### Row buffer misses per kilo instruction (RBMPKI)

In [ ]:
### READ RESULTS INTO PANDAS DATAFRAME
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

resultsdir = "../results"
# list all directories in resultsdir
configs = [d for d in os.listdir(resultsdir) if os.path.isdir(os.path.join(resultsdir, d))]
configs = ['Baseline.yaml']
# print found configs
print('Found configs: {}'.format(configs))
# list all directories under all configs
workloads = []
for c in configs:
    workloads.append([d for d in os.listdir(os.path.join(resultsdir, c)) if os.path.isdir(os.path.join(resultsdir, c, d))])
# find only the intersection of all workloads
workloads = list(set.intersection(*map(set, workloads)))
# print found workloads
print('Found workloads: {}'.format(workloads))

stats_per_config_workload = []


# for every config + workload directory
for c in configs:
    for w in workloads:
        # find all files in the directory
        files = [f for f in os.listdir(os.path.join(resultsdir, c, w)) if os.path.isfile(os.path.join(resultsdir, c, w, f))]
        # find the stats file
        stat_files = [f for f in files if f.endswith('.stats')]
        # if there is a stats file
        if stat_files:
            for stat_file in stat_files:
                # if the stats_file has less than three lines skip it
                if len(open(os.path.join(resultsdir, c, w, stat_file)).readlines()) < 3:
                    continue

                extension = ''
                # if stats_file file name itself does not start with DDR4, parse it a bit
                if not stat_file.startswith('DDR4'):
                    # get the config name from the stats_file name
                    extension = '_'.join(stat_file.split('_')[:-1])
                    # prepend underscore to extension
                    extension = '_' + extension

                # read the stats file, name columns: 'name', 'value', 'description'
                df = pd.read_csv(os.path.join(resultsdir, c, w, stat_file), header=None).T
                df.columns = df.iloc[0]
                df.drop(0,inplace=True)
                # add a new column called 'config' with the config name
                df['config'] = c + extension
                # add a new column called 'workload' with the workload name
                df['workload'] = w
                # print the stats file
                # print('Config: {}, Workload: {}, Stats: {}'.format(c, w, df))
                # append the stats to the list
                df.reset_index(inplace=True, drop=True)
                stats_per_config_workload.append(df)
        else:
            print('Config: {}, Workload: {}, Stats: No stats file found'.format(c, w))

# concatenate all stats into one dataframe
stats = pd.concat(stats_per_config_workload)

# find elements where workload does not contain '-'
# these are single core workloads
stats = stats[~stats['workload'].str.contains('-')]

# calculate row buffer misses per kilo instruction (conflicts + misses)

#record_read_conflictstotal record_read_missestotal
#record_write_conflictstotal record_write_missestotal

stats['ramulator.rbmpki'] = (stats['ramulator.row_conflicts_channel_0_core'] + stats['ramulator.row_misses_channel_0_core']) /\
                            stats['ramulator.record_insts_core_0'] * 1000
                            
stats['ramulator.mpki'] = (stats['ramulator.L3_cache_total_miss'] - stats['ramulator.L3_cache_mshr_unavailable']) /\
                            stats['ramulator.record_insts_core_0'] * 1000

# group workloads according to their rbmpki and show their names. Use three bins [0,2] [2,10] [10+]
# do not add to the dataframe, just print the results
print('MPKI per workload')
stats.sort_values(by=['ramulator.mpki'], inplace=True)

# print workload and mpki one by one

for index, row in stats.iterrows():
    print('{} = {}'.format(row['workload'], row['ramulator.mpki']))

### Motivation - Figures 2 and 3

#### OPTIONAL Pre-processing

Need to do once if you want to get per-workload sibling row activation distribution from raw data. This will take a lot of time and use a lot of memory.

We generate the raw data for this analysis using the "ACT-period-256ms.yaml" configuration.

In [ ]:
# plot cumulative_bank_usage_stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import numpy as np

# one counter per DRAM row, in each counter, there are as many entries as there are ranks + banks, which is 32
def find_distribution_bank_usage_stat(filepath, analysis_threshold):

  counters = {}

  bank_usage_stats = []

  actual_lines = []
  with open(filepath, "r") as f:
      for line in f:
          actual_lines = line.split(' ')
          break
        
  for i in range (len(actual_lines)):
    # if line is empty, skip
    if actual_lines[i] == "":
      continue
    
    line = actual_lines[i]

    tokens = line.split(":")
    ra = int(tokens[1])
    bg = int(tokens[2])
    ba = int(tokens[3])
    row = int(tokens[4])
    
    secondary_index = ra << 4 | bg << 2 | ba
    
    if row not in counters:
      init_array = []
      for i in range(32):
        init_array.append(0)
      counters[row] = init_array
      counters[row][secondary_index] += 1
    else:
      counters[row][secondary_index] += 1
      if counters[row][secondary_index] == analysis_threshold:
        # append all counter values except secondary_index to bank_usage_stats
        for j in range(len(counters[row])):
          if j != secondary_index:
            bank_usage_stats.append(counters[row][j])
               
        # remove this row from the counters
        del counters[row]
        
    # print the progress of lines so far as fraction of total lines
    if i % 1000000 == 0:
      print("Progress:", str(i / len(actual_lines)))
  
  return bank_usage_stats

# all workloads are in activate-periods directory
# under each workload directory, there is an activate_periods.txt file

MED_RBMPKI = ['510.parest', '462.libquantum', 'tpch2', 'wc_8443', 'ycsb_aserver', '473.astar', 'stream_10.trace', 'jp2_decode', '436.cactusADM', '557.xz', 'ycsb_cserver', 'ycsb_eserver', '471.omnetpp', '483.xalancbmk', '505.mcf', 'wc_map0', 'jp2_encode', 'tpch17', 'ycsb_bserver', 'tpcc64', '482.sphinx3']
HIGH_RBMPKI = ['519.lbm', '459.GemsFDTD', '450.soplex', 'h264_decode', '520.omnetpp', '433.milc', '434.zeusmp', 'random_10.trace', 'bfs_dblp', '429.mcf', '549.fotonik3d', '470.lbm', 'bfs_ny', 'bfs_cm2003', '437.leslie3d']
MEDHIGH_RBMPKI = MED_RBMPKI + HIGH_RBMPKI

# iterate over all workloads
for workload in MEDHIGH_RBMPKI:
  df = pd.DataFrame(columns=['workload', 'cumulative_bank_usage_stat', 'analysis_threshold'])
  # iterate over all analysis_thresholds
  for analysis_threshold in [2, 125, 250, 500]:
    all_bank_usage_stats = find_distribution_bank_usage_stat("activate_periods/" + workload + "/activate_periods.txt", analysis_threshold)
    workload_array = [workload for i in range(len(all_bank_usage_stats))]
    analysis_threshold_array = [analysis_threshold for i in range(len(all_bank_usage_stats))]
    # add these to df fast
    df = pd.concat([df, pd.DataFrame({'workload': workload_array, 'analysis_threshold': analysis_threshold_array, 'bank_usage_stat': all_bank_usage_stats})])    
    print("workload:", workload, "analysis_threshold:", analysis_threshold)  
  # save df as csv
  df.to_csv("distr_bank_usage_stat_" + workload + ".csv")
  

# empty dataframe with the same columns
dfn = pd.DataFrame(columns=['workload', 'cumulative_bank_usage_stat', 'analysis_threshold', 'bank_usage_stat'])

# Add RH_ESTIMATE workloads to the dataframe
for i in range (0, 32):
  dfn = pd.concat([dfn, pd.DataFrame([['ds', 0, 125]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
dfn = pd.concat([dfn, pd.DataFrame([['ds-p1', 124, 125]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 31):
  dfn = pd.concat([dfn, pd.DataFrame([['ds-p1', 0, 125]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 8):
  dfn = pd.concat([dfn, pd.DataFrame([['ds-p8', 124, 125]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 24):
  dfn = pd.concat([dfn, pd.DataFrame([['ds-p8', 0, 125]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])  
for i in range (0, 32):
  dfn = pd.concat([dfn, pd.DataFrame([['ds-p32', 124, 125]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])

for i in range (0, 32):
  dfn = pd.concat([dfn, pd.DataFrame([['ms', 0, 125]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
dfn = pd.concat([dfn, pd.DataFrame([['ms-p1', 124, 125]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 31):
  dfn = pd.concat([dfn, pd.DataFrame([['ms-p1', 0, 125]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 8):
  dfn = pd.concat([dfn, pd.DataFrame([['ms-p8', 124, 125]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 24):
  dfn = pd.concat([dfn, pd.DataFrame([['ms-p8', 0, 125]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])  
for i in range (0, 32):
  dfn = pd.concat([dfn, pd.DataFrame([['ms-p32', 124, 125]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])

# Add RH_ESTIMATE workloads to the dataframe
for i in range (0, 32):
  dfn = pd.concat([dfn, pd.DataFrame([['ds', 0, 250]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
dfn = pd.concat([dfn, pd.DataFrame([['ds-p1', 249, 250]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 31):
  dfn = pd.concat([dfn, pd.DataFrame([['ds-p1', 0, 250]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 8):
  dfn = pd.concat([dfn, pd.DataFrame([['ds-p8', 249, 250]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 24):
  dfn = pd.concat([dfn, pd.DataFrame([['ds-p8', 0, 250]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])  
for i in range (0, 32):
  dfn = pd.concat([dfn, pd.DataFrame([['ds-p32', 249, 250]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])

for i in range (0, 32):
  dfn = pd.concat([dfn, pd.DataFrame([['ms', 0, 250]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
dfn = pd.concat([dfn, pd.DataFrame([['ms-p1', 249, 250]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 31):
  dfn = pd.concat([dfn, pd.DataFrame([['ms-p1', 0, 250]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 8):
  dfn = pd.concat([dfn, pd.DataFrame([['ms-p8', 249, 250]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 24):
  dfn = pd.concat([dfn, pd.DataFrame([['ms-p8', 0, 250]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])  
for i in range (0, 32):
  dfn = pd.concat([dfn, pd.DataFrame([['ms-p32', 249, 250]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])

# Add RH_ESTIMATE workloads to the dataframe
for i in range (0, 32):
  dfn = pd.concat([dfn, pd.DataFrame([['ds', 0, 500]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
dfn = pd.concat([dfn, pd.DataFrame([['ds-p1', 499, 500]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 31):
  dfn = pd.concat([dfn, pd.DataFrame([['ds-p1', 0, 500]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 8):
  dfn = pd.concat([dfn, pd.DataFrame([['ds-p8', 499, 500]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 24):
  dfn = pd.concat([dfn, pd.DataFrame([['ds-p8', 0, 500]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])  
for i in range (0, 32):
  dfn = pd.concat([dfn, pd.DataFrame([['ds-p32', 499, 500]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])

for i in range (0, 32):
  dfn = pd.concat([dfn, pd.DataFrame([['ms', 0, 500]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
dfn = pd.concat([dfn, pd.DataFrame([['ms-p1', 499, 500]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 31):
  dfn = pd.concat([dfn, pd.DataFrame([['ms-p1', 0, 500]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 8):
  dfn = pd.concat([dfn, pd.DataFrame([['ms-p8', 499, 500]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
for i in range (0, 24):
  dfn = pd.concat([dfn, pd.DataFrame([['ms-p8', 0, 500]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])  
for i in range (0, 32):
  dfn = pd.concat([dfn, pd.DataFrame([['ms-p32', 499, 500]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])  
  
# read all csvs into one dataframe
# read one file at a time, store it in temporary dataframe
# sample 1/10 of the data randomly to put in the final datafrmae
import pandas as pd
import glob

# read all csvs into one dataframe
# read one file at a time, store it in temporary dataframe
# sample 1/10 of the data randomly to put in the final datafrmae
df = pd.DataFrame()
for filename in glob.glob("distr_bank_usage_stat_*.csv"):
    df_tmp = pd.read_csv(filename, index_col=None, header=0)
    # sample 1/10 of the data
    df_tmp = df_tmp.sample(frac=0.1, random_state=1)
    # add to final dataframe using concat
    df = pd.concat([df, df_tmp])    
    print("Done reading " + filename)

# merge dfn to df
df = pd.concat([df, dfn])

df.to_csv("distr_bank_usage.csv", index=False)


# read all csvs into one dataframe
# read one file at a time, store it in temporary dataframe
# sample 1/10 of the data randomly to put in the final datafrmae
import pandas as pd
import glob
import numpy as np
# read all csvs into one dataframe
# read one file at a time, store it in temporary dataframe
# sample 1/10 of the data randomly to put in the final datafrmae
df = pd.DataFrame()
new_df_arr = []
for filename in glob.glob("distr_bank_usage_stat_*.csv"):
    print(filename)
    df_tmp = pd.read_csv(filename, index_col=0, header=0)
    df_tmp = df_tmp[df_tmp['analysis_threshold'] == 2]
    # select bank_usage_stat as series
    sum_series = df_tmp['bank_usage_stat']
    # sum of every 31 elements
    sum_series = sum_series.groupby(np.arange(len(sum_series))//31).sum()
    # reconstruct a df from the series
    df_new = pd.DataFrame({'bank_usage_stat': sum_series})
    df_new['workload'] = df_tmp['workload'][0]
    df_new['analysis_threshold'] = df_tmp['analysis_threshold'][0]
    new_df_arr.append(df_new)
    # print size in MB of df_new
    print(df_new.memory_usage().sum() / 1024**2)

# concat new_df_arr
df = pd.concat(new_df_arr)
# merge dfn to df
df = pd.concat([df, dfn])

df.to_csv("distr_bank_usage_first_plot.csv", index=False)


#### Figure 2

In [ ]:
# plot cumulative_bank_usage_stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import numpy as np

df = pd.read_csv("../results/distr_bank_usage_first_plot.csv")

MED_RBMPKI = ['510.parest', '462.libquantum', 'tpch2', 'wc_8443', 'ycsb_aserver', '473.astar', 'stream_10.trace', 'jp2_decode', '436.cactusADM', '557.xz', 'ycsb_cserver', 'ycsb_eserver', '471.omnetpp', '483.xalancbmk', '505.mcf', 'wc_map0', 'jp2_encode', 'tpch17', 'ycsb_bserver', 'tpcc64', '482.sphinx3']
HIGH_RBMPKI = ['519.lbm', '459.GemsFDTD', '450.soplex', 'h264_decode', '520.omnetpp', '433.milc', '434.zeusmp', 'bfs_dblp', '429.mcf', '549.fotonik3d', 'random_10.trace', 'gups', '470.lbm', 'bfs_ny', 'bfs_cm2003', '437.leslie3d']
RH_ESTIMATE = ['ds', 'ds-p1' , 'ds-p8', 'ds-p32', 'ms' , 'ms-p1', 'ms-p8', 'ms-p32']
# dfx = df.copy()
dfx = df



# Add RH_ESTIMATE workloads to the dataframe
dfx = pd.concat([dfx, pd.DataFrame([['ds', 0, 2]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
dfx = pd.concat([dfx, pd.DataFrame([['ds-p1', 1, 2]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
dfx = pd.concat([dfx, pd.DataFrame([['ds-p8', 8, 2]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
dfx = pd.concat([dfx, pd.DataFrame([['ds-p32', 31, 2]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
dfx = pd.concat([dfx, pd.DataFrame([['ms', 0, 2]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
dfx = pd.concat([dfx, pd.DataFrame([['ms-p1', 1, 2]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
dfx = pd.concat([dfx, pd.DataFrame([['ms-p8', 8, 2]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])
dfx = pd.concat([dfx, pd.DataFrame([['ms-p32', 31, 2]], columns=['workload', 'bank_usage_stat', 'analysis_threshold'])])

MEDHIGH_RBMPKI = MED_RBMPKI + HIGH_RBMPKI + RH_ESTIMATE
# remove where workload is stream_10.trace and random_10.trace from MEDHIGH_RBMPKI
MEDHIGH_RBMPKI.remove('stream_10.trace')
MEDHIGH_RBMPKI.remove('gups')
# MEDHIGH_RBMPKI.remove('random_10.trace')
# get rid of workloads not in MEDHIGH_RBMPKI
dfn = dfx[dfx['workload'].isin(MEDHIGH_RBMPKI)]
# dfn['use_stat'] = dfn['cumulative_bank_usage_stat'] * 32
# remove where workload is stream_10.trace and random_10.trace
dfn = dfn[dfn['workload'] != 'stream_10.trace']
# dfn = dfn[dfn['workload'] != 'random_10.trace']
dfn = dfn[dfn['workload'] != 'gups']

dfn['workload'] = dfn['workload'].replace('random_10.trace', 'gups')

# #rename random_10.trace to gups in MEDHIGH_RBMPKI
MEDHIGH_RBMPKI = [x.replace('random_10.trace', 'gups') for x in MEDHIGH_RBMPKI]
order = ['h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver', 'ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'stream_10.trace', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp', 'ds', 'ds-p1' , 'ds-p8', 'ds-p32', 'ms' , 'ms-p1', 'ms-p8', 'ms-p32']
# remove from order, workloads not in MEDHIGH_RBMPKI
order = [x for x in order if x in MEDHIGH_RBMPKI]
# sort by workload according to order
dfn['workload'] = pd.Categorical(dfn['workload'], order)
dfn = dfn.sort_values('workload')

sns.set_theme(style="whitegrid")
# create side-by-side two subplots
fig, (ax1) = plt.subplots(ncols=1, figsize=(7, 1.5))

PROPS = {
    'boxprops':{'edgecolor':'black'},
    'medianprops':{'color':'black'},
    'whiskerprops':{'color':'black'},
    'capprops':{'color':'black'}
}


flierprops = dict(markerfacecolor='black', markeredgecolor='black', markersize=2,
              linestyle='none')

# barplot workload on x axis, cumulative_bank_usage_stat on y axis, only for analysis_threshold = 4, only for workloads with high RBMPKI
sns.boxplot(ax=ax1, x="workload", y="bank_usage_stat", data=dfn[(dfn['analysis_threshold'] == 2) & (dfn['workload'].isin(MEDHIGH_RBMPKI))], palette="pastel", showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"3"}, flierprops=flierprops,**PROPS)
# rotate x axis labels
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90)
# rename x axis
ax1.set_xlabel("Workload")
# rename y axis
ax1.set_ylabel("Number of\nsibling row activations")


# show y until 4 for ax1
ax1.set_ylim([0, 35.84])
# y ticks in increments of 1 for ax1
ax1.set_yticks(np.arange(0, 32, 8).tolist() + [31]) 
# draw a red line at 4
ax1.axhline(y=31, color='r', linestyle='-')
# move y axis labels down
ax1.yaxis.set_label_coords(-0.08,0.1)

# make x axis tick labels smaller
ax1.tick_params(axis='x', labelsize=10)

# draw verticel line between leslie3d and ds
ax1.axvline(x=34.5, color='black', linestyle='--')
# continue the line below drawing area

# add text to the right of the vertical line
ax1.text(38.7, -30, 'RowHammer\nAttacks', fontsize=9, va='center', ha='center', color='brown')

# color the last 8 x axis tick labels blue
for tick in ax1.get_xticklabels()[-8:]:
  tick.set_color('brown')


# save as pdf tight layout
plt.savefig("sibling_row_activations.pdf", bbox_inches='tight')

# print average use_stat across workloads
# print("Average use_stat across workloads:", dfn[(dfn['analysis_threshold'] == 2)]['use_stat'].mean())
# # max and min
# print("Max use_stat across workloads:", dfn[(dfn['analysis_threshold'] == 2)]['use_stat'].max())
# print("Min use_stat across workloads:", dfn[(dfn['analysis_threshold'] == 2)]['use_stat'].min())

#### Figure 3

In [ ]:
# plot cumulative_bank_usage_stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import numpy as np

df = pd.read_csv("../results/distr_bank_usage.csv")

MED_RBMPKI = ['510.parest', '462.libquantum', 'tpch2', 'wc_8443', 'ycsb_aserver', '473.astar', 'stream_10.trace', 'jp2_decode', '436.cactusADM', '557.xz', 'ycsb_cserver', 'ycsb_eserver', '471.omnetpp', '483.xalancbmk', '505.mcf', 'wc_map0', 'jp2_encode', 'tpch17', 'ycsb_bserver', 'tpcc64', '482.sphinx3']
HIGH_RBMPKI = ['519.lbm', '459.GemsFDTD', '450.soplex', 'h264_decode', '520.omnetpp', '433.milc', '434.zeusmp', 'bfs_dblp', '429.mcf', '549.fotonik3d', 'random_10.trace', '470.lbm', 'bfs_ny', 'bfs_cm2003', '437.leslie3d']
MEDHIGH_RBMPKI = MED_RBMPKI + HIGH_RBMPKI
RH_ESTIMATE = ['ds', 'ds-p1' , 'ds-p8', 'ds-p32', 'ms' , 'ms-p1', 'ms-p8', 'ms-p32']

MEDHIGH_RBMPKI = MED_RBMPKI + HIGH_RBMPKI + RH_ESTIMATE

# remove where workload is stream_10.trace and random_10.trace from MEDHIGH_RBMPKI
MEDHIGH_RBMPKI.remove('stream_10.trace')
# MEDHIGH_RBMPKI.remove('random_10.trace')
# MEDHIGH_RBMPKI.remove('gups')
# get rid of workloads not in MEDHIGH_RBMPKI
dfn = df[df['workload'].isin(MEDHIGH_RBMPKI)].copy()
# remove where workload is stream_10.trace and random_10.trace
dfn = dfn[dfn['workload'] != 'stream_10.trace']
# dfn = dfn[dfn['workload'] != 'random_10.trace']
dfn = dfn[dfn['workload'] != 'gups']

# rename random_10.trace to gups
dfn['workload'] = dfn['workload'].replace('random_10.trace', 'gups')

#rename random_10.trace to gups in MEDHIGH_RBMPKI
MEDHIGH_RBMPKI = [x.replace('random_10.trace', 'gups') for x in MEDHIGH_RBMPKI]

order = ['h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver', 'ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'stream_10.trace', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp', 'ds', 'ds-p1' , 'ds-p8', 'ds-p32', 'ms' , 'ms-p1', 'ms-p8', 'ms-p32']
# remove from order, workloads not in MEDHIGH_RBMPKI
order = [x for x in order if x in MEDHIGH_RBMPKI]
# sort by workload according to order
dfn['workload'] = pd.Categorical(dfn['workload'], order)
dfn = dfn.sort_values('workload')

sns.set_theme(style="whitegrid")
# create side-by-side two subplots
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, figsize=(7, 4.5), sharex=True)

PROPS = {
    'boxprops':{'edgecolor':'black'},
    'medianprops':{'color':'black'},
    'whiskerprops':{'color':'black'},
    'capprops':{'color':'black'}
}

flierprops = dict(markerfacecolor='black', markeredgecolor='black', markersize=2,
              linestyle='none')

# barplot workload on x axis, cumulative_bank_usage_stat on y axis, only for analysis_threshold = 4, only for workloads with high RBMPKI
sns.boxplot(ax=ax3, x="workload", y="bank_usage_stat", data=dfn[(dfn['analysis_threshold'] == 125) & (dfn['workload'].isin(MEDHIGH_RBMPKI))], palette="pastel", showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"3"}, flierprops=flierprops,**PROPS)
# rotate x axis labels

# rename x axis
ax3.set_xlabel("Workload")

# rename y axis
ax3.set_ylabel("Average Counter Value")

# plot the same but for analysis_threshold = 32
sns.boxplot(ax=ax2, x="workload", y="bank_usage_stat", data=dfn[(dfn['analysis_threshold'] == 250) & (dfn['workload'].isin(MEDHIGH_RBMPKI))], palette="pastel", showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"3"},flierprops=flierprops, **PROPS)
# rotate x axis labels
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=90)
ax2.set_xlabel("")
ax2.set_ylabel("Average Counter Value")


# plot the same but for analysis_threshold = 32
sns.boxplot(ax=ax1, x="workload", y="bank_usage_stat", data=dfn[(dfn['analysis_threshold'] == 500) & (dfn['workload'].isin(MEDHIGH_RBMPKI))], palette="pastel", showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"3"},flierprops=flierprops, **PROPS)
# rotate x axis labels
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90)
ax1.set_xlabel("")
ax1.set_ylabel("Sibling row activation count")

ax3.set_xlabel("Workload")
ax3.set_xticklabels(ax3.get_xticklabels(), rotation=90)

# set x axis tick labels using workload names
ax3.set_xticklabels(order)


# show y until 4 for ax1
ax3.set_ylim([0, 140])
# y ticks in increments of 1 for ax1
ax3.set_yticks(np.arange(0, 125, 32).tolist() + [125])
# draw a red line at 4
ax3.axhline(y=125, color='r', linestyle='-')
ax2.set_ylim([0, 280])
# y ticks in increments of 1 for ax1
ax2.set_yticks(np.arange(0, 249, 64).tolist() + [250])
# draw a red line at (analysis_threshold-1)
ax2.axhline(y=250, color='r', linestyle='-')
# show y until 4 for ax1
ax1.set_ylim([0, 560])
# y ticks in increments of 1 for ax1
ax1.set_yticks(np.arange(0, 500, 128).tolist() + [500])
# draw a red line at (analysis_threshold-1)
ax1.axhline(y=500, color='r', linestyle='-')

# show y until 4 for ax1

# move y axis labels down
ax1.yaxis.set_label_coords(-0.12,-0.8)
ax2.yaxis.set_label_coords(-0.09,0.3)
ax3.yaxis.set_label_coords(-0.09,0.3)

# make x axis tick labels smaller
ax1.tick_params(axis='x', labelsize=10)
ax2.tick_params(axis='x', labelsize=10)
ax3.tick_params(axis='x', labelsize=10)

# remove ax2-3 y axis labels
ax2.set_ylabel('')
ax3.set_ylabel('')


# title each ax
ax3.set_title("RowHammer Threshold $(N_{RH})$ = 125")
ax2.set_title("RowHammer Threshold $(N_{RH})$ = 250")
ax1.set_title("RowHammer Threshold $(N_{RH})$ = 500")

# draw verticel line between leslie3d and ds
ax1.axvline(x=34.5, color='black', linestyle='--')
ax2.axvline(x=34.5, color='black', linestyle='--')
ax3.axvline(x=34.5, color='black', linestyle='--')
# continue the line below drawing area
# add text to the right of the vertical line
ax3.text(38.7, -140, 'RowHammer\nAttacks', fontsize=9, va='center', ha='center', color='brown')

# color the last 8 x axis tick labels blue
for tick in ax1.get_xticklabels()[-8:]:
  tick.set_color('brown')
# color the last 8 x axis tick labels blue
for tick in ax2.get_xticklabels()[-8:]:
  tick.set_color('brown')
# color the last 8 x axis tick labels blue
for tick in ax3.get_xticklabels()[-8:]:
  tick.set_color('brown')

# reduce gap between subplots
plt.subplots_adjust(hspace=0.3)

# save as pdf tight layout
plt.savefig("distr_counter_values.pdf", bbox_inches='tight')

### Performance and Energy Evaluation

#### Figure 8 and 16

In [ ]:
MY_MECHANISM_NAME = 'ABACuS'

### READ RESULTS INTO PANDAS DATAFRAME
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

resultsdir = "../results"
# list all directories in resultsdir
configs = [d for d in os.listdir(resultsdir) if os.path.isdir(os.path.join(resultsdir, d))]
configs = ['Baseline.yaml', 'Hydra-Baseline.yaml', 'REGA125.yaml', 'REGA250.yaml', 'REGA500.yaml', 'REGA1000.yaml', 'Graphene125.yaml', 'Graphene250.yaml', 'Graphene500.yaml', 'Graphene1000.yaml', 'Hydra125.yaml', 'Hydra250.yaml', 'Hydra500.yaml', 'Hydra1000.yaml', 'PARA125.yaml', 'PARA250.yaml', 'PARA500.yaml', 'PARA1000.yaml', 
           'ABACUS125.yaml', 'ABACUS250.yaml', 'ABACUS500.yaml', 'ABACUS1000.yaml', 'ABACUS125-Big.yaml', 'ABACUS250-Big.yaml', 'ABACUS500-Big.yaml', 'ABACUS1000-Big.yaml']
# print found configs
print('Found configs: {}'.format(configs))
# list all directories under all configs
workloads = []
for c in configs:
    workloads.append([d for d in os.listdir(os.path.join(resultsdir, c)) if os.path.isdir(os.path.join(resultsdir, c, d))])
# find only the intersection of all workloads
workloads = list(set.intersection(*map(set, workloads)))
# print found workloads
print('Found workloads: {}'.format(workloads))

stats_per_config_workload = []
workloads = [w for w in workloads if not '-' in w]

# for every config + workload directory
for c in configs:
    for w in workloads:
        # find all files in the directory
        files = [f for f in os.listdir(os.path.join(resultsdir, c, w)) if os.path.isfile(os.path.join(resultsdir, c, w, f))]
        # find the stats file
        stat_files = [f for f in files if f.endswith('.stats')]
        # if there is a stats file
        if stat_files:
            for stat_file in stat_files:
                # if the stats_file has less than three lines skip it
                if len(open(os.path.join(resultsdir, c, w, stat_file)).readlines()) < 3:
                    continue
                
                # print the name of the stats_file
                print('Found stats file: {}'.format(os.path.join(os.path.join(resultsdir, c, w, stat_file))))

                extension = ''
                # if stats_file file name itself does not start with DDR4, parse it a bit
                if not stat_file.startswith('DDR4'):
                    # get the config name from the stats_file name
                    extension = '_'.join(stat_file.split('_')[:-1])
                    # prepend underscore to extension
                    extension = '_' + extension

                # read the stats file, name columns: 'name', 'value', 'description'
                df = pd.read_csv(os.path.join(resultsdir, c, w, stat_file), header=None).T
                df.columns = df.iloc[0]
                df.drop(0,inplace=True)
                # add a new column called 'config' with the config name
                df['config'] = c + extension
                # add a new column called 'workload' with the workload name
                df['workload'] = w
                # print the stats file
                # print('Config: {}, Workload: {}, Stats: {}'.format(c, w, df))
                # append the stats to the list
                df.reset_index(inplace=True, drop=True)
                stats_per_config_workload.append(df)
        else:
            print('Config: {}, Workload: {}, Stats: No stats file found'.format(c, w))

# concatenate all stats into one dataframe
stats = pd.concat(stats_per_config_workload)

# find elements where workload does not contain '-'
# these are multi core workloads
stats = stats[~stats['workload'].str.contains('-')]

# remove these two workloads: stream_10.trace and random_10.trace
stats = stats[~stats['workload'].isin(['gups'])]
# also from workloads
workloads = [w for w in workloads if not w in ['gups']]

# make sure config is string
stats['config'] = stats['config'].astype(str)

# remove "-16DR" from config names
stats['config'] = stats['config'].str.replace('-16DR', '')

# replace 1K with 1000 in config names
stats['config'] = stats['config'].str.replace('1K', '1000')

# replace 'Baseline' with 'Baseline0'
stats['config'] = stats['config'].str.replace('Baseline', 'Baseline0')

# add a new column that stores in integer the number in the config name
stats['nrh'] = stats['config'].str.extract('(\d+)').astype(int)

# remove numbers from config names
stats['config'] = stats['config'].str.replace('\d+', '', regex=True)

# remove yaml from config names
stats['config'] = stats['config'].str.replace('.yaml', '')

# replace SPR with APAR
stats['config'] = stats['config'].str.replace('ABACUS', MY_MECHANISM_NAME)

stats.loc[stats['workload'] == 'random_10.trace', 'workload'] = 'gups'

# increasing order of rbmpki
# order = ['511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', '500.perlbench', '523.xalancbmk', '510.parest', '557.xz', '482.sphinx3', '505.mcf', '436.cactusADM', '471.omnetpp', '473.astar', '483.xalancbmk', '462.libquantum', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf']
order = ['h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver', 'ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'stream_10.trace', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp']

# remove all workloads not in order
stats = stats[stats['workload'].isin(order)]
# also from the workload list
workloads = [w for w in workloads if w in order]

# order workloads according to the order
stats['workload'] = pd.Categorical(stats['workload'], categories=order, ordered=True)

stats_copy = stats.copy()

# import MultipleLocator
from matplotlib.ticker import MultipleLocator

# use seaborn-deep style
sns.set(font_scale=1.0)
sns.set_style("whitegrid")
sns.set_palette("pastel", n_colors=5)

stats = stats_copy.copy()

# instructions per cycle (IPC) is record_cycles_insts_0 / record_cycs_core_0
stats['ramulator.ipc'] = stats['ramulator.record_insts_core_0'] / stats['ramulator.record_cycs_core_0']


stats['ramulator.rbmpki'] = (stats['ramulator.row_conflicts_channel_0_core'] + stats['ramulator.row_misses_channel_0_core']) /\
                            stats['ramulator.record_insts_core_0'] * 1000

# copy the IPC of the baseline config as to all configs
baseline = stats[stats['config'] == 'Baseline']
baseline = baseline[['workload', 'ramulator.ipc', 'ramulator.read_latency_avg_0', 'ramulator.rbmpki', 'ramulator.window_full_stall_cycles_core_0']]
# baseline
baseline.columns = ['workload', 'ramulator.baseline_ipc', 'ramulator.baseline_read_latency_avg_0', 'ramulator.baseline_rbmpki', 'ramulator.baseline_stall_cycles']
stats = pd.merge(stats, baseline, on='workload')
#hydra baseline
hydra_baseline = stats[stats['config'] == 'Hydra-Baseline']
hydra_baseline = hydra_baseline[['workload', 'ramulator.ipc']]
# hydra_baseline
hydra_baseline.columns = ['workload', 'ramulator.hydra_baseline_ipc']
stats = pd.merge(stats, hydra_baseline, on='workload')

stats['ramulator.normalized_ipc'] = stats['ramulator.ipc'] / stats['ramulator.baseline_ipc']
stats['ramulator.normalized_read_latency'] = stats['ramulator.read_latency_avg_0'] / stats['ramulator.baseline_read_latency_avg_0']
stats['ramulator.normalized_stall_cycles'] = stats['ramulator.window_full_stall_cycles_core_0'] / stats['ramulator.baseline_stall_cycles']
stats['ramulator.normalized_rbmpki'] = stats['ramulator.rbmpki'] / stats['ramulator.baseline_rbmpki']

# normalized ipc for hydra is not correct, so we overwrite it with the correct value
stats.loc[stats['config'].str.contains('Hydra'), 'ramulator.normalized_ipc'] = stats['ramulator.ipc'] / stats['ramulator.hydra_baseline_ipc']

# add the geometric normalized ipc average as a new workload to every config
geometric_mean = stats.groupby(['config','nrh'])['ramulator.normalized_ipc'].apply(lambda x: x.prod()**(1.0/len(x))).reset_index()
geometric_mean['workload'] = 'GeoMean'



stats = pd.concat([stats, geometric_mean])

# order in decreasing nRH (the nRH column) use pd.Categorical on nrh
stats['nrh'] = pd.Categorical(stats['nrh'], categories=[1000, 500, 250, 125], ordered=True)


# order = ['GeoMean', '531.deepsjeng', '502.gcc', '541.leela', '435.gromacs', '481.wrf', '458.sjeng', '445.gobmk', '444.namd', '508.namd', '401.bzip2', '456.hmmer', '403.gcc', '464.h264ref', '526.blender', '447.dealII', '544.nab', '523.xalancbmk', '500.perlbench', '538.imagick', '525.x264', '507.cactuBSSN', '511.povray', '462.libquantum', '473.astar', '510.parest', '482.sphinx3', '505.mcf', '557.xz', '471.omnetpp', '483.xalancbmk', '436.cactusADM', '520.omnetpp', '450.soplex', '470.lbm', '519.lbm', '434.zeusmp', '433.milc', '459.GemsFDTD', '549.fotonik3d', '429.mcf', '437.leslie3d']
order = ['h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver', 'ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp','GeoMean']

# order = ['GeoMean', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', '500.perlbench', '523.xalancbmk', '510.parest', '557.xz', '482.sphinx3', '505.mcf', '436.cactusADM', '471.omnetpp', '473.astar', '483.xalancbmk', '462.libquantum', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf']

stats['workload'] = pd.Categorical(stats['workload'], categories=order, ordered=True)

#barplot of normalized IPC, also draw edges around bars
fig, ax = plt.subplots(figsize=(13, 4))
ax = sns.barplot(x='workload', y='ramulator.normalized_ipc', hue='nrh', data=stats[(stats['config'] == 'ABACuS')], edgecolor='black', linewidth=0.5)


ax.set_xlabel('Workload')
ax.set_ylabel('Normalized IPC')
# move ylabel down
ax.yaxis.set_label_coords(-0.045,0.45)
# draw a red line at y = 1.0, label it as baseline IPC
ax.axhline(y=1.0, color='r', linestyle='--')
# write above the red line 'baseline IPC' using the same pastel red color
ax.text(0.01, 0.7, 'Baseline IPC', color='#e74c3c', transform=ax.transAxes, fontsize=15)
# extend the y axis to 1.2
ax.set_ylim(0.6, 1.2)
# color the 5th y tick red
ax.get_yticklabels()[2].set_color('#e74c3c')
# rotate x axis ticks
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
# make axis tick font bigger
ax.tick_params(axis='both', which='major', labelsize=11)
ax.tick_params(axis='y', which='major', labelsize=12)

# # draw vertical lines to separate the rowhammer threshold values
ax.axvline(x=26.52, color='grey', linestyle='-', alpha=0.5)
# put text before the line saying "LOW RBMPKI"
ax.text(0.33, 0.7, 'LOW RBMPKI', color='grey', transform=ax.transAxes, fontsize=12)
# put arrow to the left above text
ax.annotate('', xy=(19.5, 1.08), xytext=(25.5, 1.08), arrowprops=dict(facecolor='grey', shrink=0.01, width=3, headwidth=10, alpha=0.5))
ax.axvline(x=46.52, color='grey', linestyle='-', alpha=0.5)
ax.text(0.65, 0.7, 'MED. RBMPKI', color='grey', transform=ax.transAxes, fontsize=12)
ax.annotate('', xy=(39.5, 1.08), xytext=(45.5, 1.08), arrowprops=dict(facecolor='grey', shrink=0.01, width=3, headwidth=10, alpha=0.5))
ax.axvline(x=61.52, color='grey', linestyle='-', alpha=0.5)
ax.text(0.875, 0.7, 'HIGH RBMPKI', color='grey', transform=ax.transAxes, fontsize=12)
ax.annotate('', xy=(53.5, 1.08), xytext=(60.5, 1.08), arrowprops=dict(facecolor='grey', shrink=0.01, width=3, headwidth=10, alpha=0.5))

# make x and y axis labels bigger
ax.xaxis.label.set_size(16)
#ax.yaxis.label.set_fontweight('bold')
ax.yaxis.label.set_size(16)
#ax.xaxis.label.set_fontweight('bold')

# put the legend on top of the plot
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=4, fancybox=True, shadow=True, fontsize=12)
# prepend "nRH" to legend names
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ['$N_{RH}$ = ' + label for label in labels], loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=4, fancybox=True, shadow=True, fontsize=12)

# highlight the geometric mean ax label
ax.get_xticklabels()[62].set_fontweight('bold')

plt.tight_layout()
plt.show()

# save figure
fig.savefig('abacus_performance_single_core.pdf', bbox_inches='tight')
# export data to csv
stats.to_csv('abacus_performance_single_core.csv', index=False)

# numbers to put in paper

# ABACuS at 1000 nRH normalized_ipc for geomean workload
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 1000) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'])
# ABACuS at 1000 nRH minimum normalized_ipc
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 1000)]['ramulator.normalized_ipc'].min())
# ABACuS at 1000 nRH preventive_refreshes_channel_0_core average
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 1000)]['ramulator.preventive_refreshes_channel_0_core'].mean())
# ABACuS at 1000 nRH normalized_read_latency average
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 1000)]['ramulator.normalized_read_latency'].mean())

# above stats for nrh = 125
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'])
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125)]['ramulator.normalized_ipc'].min())
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125)]['ramulator.preventive_refreshes_channel_0_core'].mean())
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125)]['ramulator.normalized_read_latency'].mean())

# REGA at 125 nrh normalized_ipc for geomean workload
print(stats[(stats['config'] == 'REGA') & (stats['nrh'] == 125) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'])

# PARA at 125 nrh normalized_ipc for geomean workload
print(stats[(stats['config'] == 'PARA') & (stats['nrh'] == 125) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'])

# PARA at 1000 nrh normalized_ipc for geomean workload
print(stats[(stats['config'] == 'PARA') & (stats['nrh'] == 1000) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'])

# Hydra at 125 nrh normalized_ipc for geomean workload divided by Abacus at 125 nrh normalized_ipc for geomean workload
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'].values[0]/ stats[(stats['config'] == 'Hydra') & (stats['nrh'] == 125) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'].values[0])
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 250) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'].values[0]/ stats[(stats['config'] == 'Hydra') & (stats['nrh'] == 250) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'].values[0])
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 500) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'].values[0]/ stats[(stats['config'] == 'Hydra') & (stats['nrh'] == 500) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'].values[0])
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 1000) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'].values[0]/ stats[(stats['config'] == 'Hydra') & (stats['nrh'] == 1000) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'].values[0])


# Hydra normalized RBMPKI at nrh = 125 for geomean
print('hydravsabacus rbmpki: ', stats[(stats['config'] == 'Hydra') & (stats['nrh'] == 125)]['ramulator.normalized_rbmpki'].mean()- stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125)]['ramulator.normalized_rbmpki'].mean())
# Hydra at 125 nRH normalized_read_latency average
print('hydravsabacus latency: ', stats[(stats['config'] == 'Hydra') & (stats['nrh'] == 125)]['ramulator.normalized_read_latency'].mean() - stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125)]['ramulator.normalized_read_latency'].mean())

# mean number of preventive refreshes for ABACuS at 125 nrh
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125)]['ramulator.preventive_refreshes_channel_0_core'].mean())
print(stats[(stats['config'] == 'Graphene') & (stats['nrh'] == 125)]['ramulator.preventive_refreshes_channel_0_core'].mean())

# mean normalized_stall_cycles for ABACuS at 125 nrh
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125)]['ramulator.normalized_stall_cycles'].mean())
print(stats[(stats['config'] == 'Graphene') & (stats['nrh'] == 125)]['ramulator.normalized_stall_cycles'].mean())

evaluation_paragraph = """
ABACuS average slowdown at 1K nRH {avg_slowdown_1k}
ABACuS max slowdown at 1K nRH {max_slowdown_1k}
ABACuS average memory latency increase at 1K nRH {avg_mem_lat_1k}

ABACuS average slowdown at 125 nRH {avg_slowdown_125}
ABACuS max slowdown at 125 nRH {max_slowdown_125}
ABACuS average memory latency increase at 125 nRH {avg_mem_lat_125}
""".format(
    avg_slowdown_1k=1-(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 1000)]['ramulator.normalized_ipc'].mean()),
    max_slowdown_1k=1-(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 1000)]['ramulator.normalized_ipc'].min()),
    avg_mem_lat_1k=stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 1000)]['ramulator.normalized_read_latency'].mean(),
    avg_slowdown_125=1-(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125)]['ramulator.normalized_ipc'].mean()),
    max_slowdown_125=1-(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125)]['ramulator.normalized_ipc'].min()),
    avg_mem_lat_125=stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125)]['ramulator.normalized_read_latency'].mean(),
)

print(evaluation_paragraph)


# Graphene at 125 nrh normalized_ipc for geomean workload divided by Abacus at 125 nrh normalized_ipc for geomean workload
# print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'].values[0]/ stats[(stats['config'] == 'Graphene') & (stats['nrh'] == 125) & (stats['workload'] == 'GeoMean')]['ramulator.normalized_ipc'].values[0])

# use seaborn-deep style
sns.set(font_scale=1.0)
sns.set_style("whitegrid")
sns.set_palette("pastel", n_colors=1)

stats = stats_copy.copy()

# instructions per cycle (IPC) is record_cycles_insts_0 / record_cycs_core_0
stats['ramulator.ipc'] = stats['ramulator.record_insts_core_0'] / stats['ramulator.record_cycs_core_0']


stats['ramulator.rbmpki'] = (stats['ramulator.row_conflicts_channel_0_core'] + stats['ramulator.row_misses_channel_0_core']) /\
                            stats['ramulator.record_insts_core_0'] * 1000

# copy the IPC of the baseline config as to all configs
baseline = stats[stats['config'] == 'Baseline']
baseline = baseline[['workload', 'ramulator.ipc', 'ramulator.read_latency_avg_0', 'ramulator.rbmpki', 'ramulator.window_full_stall_cycles_core_0']]
# baseline
baseline.columns = ['workload', 'ramulator.baseline_ipc', 'ramulator.baseline_read_latency_avg_0', 'ramulator.baseline_rbmpki', 'ramulator.baseline_stall_cycles']
stats = pd.merge(stats, baseline, on='workload')
#hydra baseline
hydra_baseline = stats[stats['config'] == 'Hydra-Baseline']
hydra_baseline = hydra_baseline[['workload', 'ramulator.ipc']]
# hydra_baseline
hydra_baseline.columns = ['workload', 'ramulator.hydra_baseline_ipc']
stats = pd.merge(stats, hydra_baseline, on='workload')

# order and sort nRH high to low
stats['nrh'] = pd.Categorical(stats['nrh'], categories=[1000, 500, 250, 125], ordered=True)
stats = stats.sort_values('nrh')

stats['ramulator.normalized_ipc'] = stats['ramulator.ipc'] / stats['ramulator.baseline_ipc']
stats['ramulator.normalized_read_latency'] = stats['ramulator.read_latency_avg_0'] / stats['ramulator.baseline_read_latency_avg_0']
stats['ramulator.normalized_stall_cycles'] = stats['ramulator.window_full_stall_cycles_core_0'] / stats['ramulator.baseline_stall_cycles']
stats['ramulator.normalized_rbmpki'] = stats['ramulator.rbmpki'] / stats['ramulator.baseline_rbmpki']

# normalized ipc for hydra is not correct, so we overwrite it with the correct value
stats.loc[stats['config'].str.contains('Hydra'), 'ramulator.normalized_ipc'] = stats['ramulator.ipc'] / stats['ramulator.hydra_baseline_ipc']

# add the geometric normalized ipc average as a new workload to every config
# geometric_mean = stats.groupby(['config','nrh'])['ramulator.normalized_ipc'].apply(lambda x: x.prod()**(1.0/len(x))).reset_index()
# geometric_mean['workload'] = 'GeoMean'

# stats = pd.concat([stats, geometric_mean])

# # order = ['GeoMean', '531.deepsjeng', '502.gcc', '541.leela', '435.gromacs', '481.wrf', '458.sjeng', '445.gobmk', '444.namd', '508.namd', '401.bzip2', '456.hmmer', '403.gcc', '464.h264ref', '526.blender', '447.dealII', '544.nab', '523.xalancbmk', '500.perlbench', '538.imagick', '525.x264', '507.cactuBSSN', '511.povray', '462.libquantum', '473.astar', '510.parest', '482.sphinx3', '505.mcf', '557.xz', '471.omnetpp', '483.xalancbmk', '436.cactusADM', '520.omnetpp', '450.soplex', '470.lbm', '519.lbm', '434.zeusmp', '433.milc', '459.GemsFDTD', '549.fotonik3d', '429.mcf', '437.leslie3d']
# order = ['GeoMean', 'h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver', 'ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp']

low_rbmpki = ['h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver']
med_rbmpki = ['ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2']
high_rbmpki = ['433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp']

# add new column called rbmpki_category, set category if workload matches one of the above lists
stats['rbmpki_category'] = np.where(stats['workload'].isin(low_rbmpki), 'low (<2)', np.where(stats['workload'].isin(med_rbmpki), 'medium (<10)', np.where(stats['workload'].isin(high_rbmpki), 'high (>10)', 'none')))

# geometric_mean = stats.groupby(['config','nrh','rbmpki_category'])['ramulator.normalized_ipc'].apply(lambda x: x.prod()**(1.0/len(x))).reset_index()
# geometric_mean['workload'] = 'GeoMean'
# stats = pd.concat([stats, geometric_mean])

# remove rows where rbmpki_category is none
stats = stats[stats['rbmpki_category'] != 'none']

order = ['low (<2)', 'medium (<10)', 'high (>10)']
stats['rbmpki_category'] = pd.Categorical(stats['rbmpki_category'], categories=order, ordered=True)

#barplot of normalized IPC, also draw edges around bars
fig, ax = plt.subplots(figsize=(7, 2))
ax = sns.boxplot(x='rbmpki_category', y='ramulator.normalized_ipc', hue='nrh', data=stats[(stats['config'] == 'ABACuS')], showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black"})

ax.set_xlabel('Row buffer misses per kilo instructions (RBMPKI)')
ax.set_ylabel('Normalized IPC\ndistribution')
ax.axhline(y=1.0, color='#e74c3c', linestyle='--')
# write above the red line 'baseline IPC' using the same pastel red color
ax.text(0.01, 0.85, 'Baseline IPC', color='#e74c3c', transform=ax.transAxes, fontsize=15)
# extend the y axis to 1.2
ax.set_ylim(0.6, 1.1)
# rotate x axis ticks
ax.set_xticklabels(ax.get_xticklabels())
# make axis tick font bigger
ax.tick_params(axis='both', which='major', labelsize=11)
ax.tick_params(axis='y', which='major', labelsize=12)

# have four y axis ticks
ax.set_yticks([0.6, 0.7, 0.8, 0.9, 1.0, 1.1])

# color the 5th y tick red
ax.get_yticklabels()[4].set_color('#e74c3c')
# # make x and y axis labels bigger
ax.xaxis.label.set_size(16)
# ax.yaxis.label.set_fontweight('bold')
ax.yaxis.label.set_size(16)
# ax.xaxis.label.set_fontweight('bold')

# draw a circle around the minimum value of abacus at nrh = 1000
# get the minimum value of abacus at nrh = 1000
ax.add_patch(plt.Circle((2.3, 0.87), 0.03, color='blue', fill=False, clip_on=False))
ax.add_patch(plt.Circle((2.1, 0.75), 0.03, color='blue', fill=False, clip_on=False))
ax.add_patch(plt.Circle((1.9, 0.725), 0.03, color='blue', fill=False, clip_on=False))
ax.add_patch(plt.Circle((1.7, 0.677), 0.03, color='blue', fill=False, clip_on=False))
# write the minimum value of abacus at nrh = 1000
ax.text(1.59, 0.8, 'gups', color='blue', fontsize=12, bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.2', alpha = 0.95))
# draw an arrow from text to circle
# ax.annotate("", xy=(-0.32, 0.88), xytext=(-0.2, 0.47), arrowprops=dict(arrowstyle="->", color='blue', lw=1))


# # put the legend on top of the plot
# ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=4, fancybox=True, shadow=True, fontsize=10)
# # prepend "nRH" to legend names
handles, labels = ax.get_legend_handles_labels()
newlabels = ['$N_{RH}$ = ' + label for label in labels]
newlabels[0] += ' (leftmost box)'
newlabels[3] += ' (rightmost box)'
ax.legend(handles, newlabels, loc='upper center', bbox_to_anchor=(0.235, 0.74), ncol=1, fancybox=True, shadow=True, fontsize=10)

# remove legend
ax.legend_.remove()

ax.annotate('$N_{RH}=1000$ (leftmost box)\n$N_{RH}=500$\n$N_{RH}=250$\n$N_{RH}=125$ (rightmost box)', xy=(0.05, 0.60), xycoords='axes fraction',
            size=10, ha='left', va='top',
            bbox=dict(boxstyle='round', fc='w', ec='k'))


# plt.tight_layout()
plt.show()

# save figure
fig.savefig('abacus_performance_single_core_small.pdf', bbox_inches='tight')
# export data to csv
stats.to_csv('abacus_performance_single_core_small.csv', index=False)

# ABACuS average normalized IPC at nRH = 1000
print(1-stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 1000)]['ramulator.normalized_ipc'].mean())
# ABACuS min. normalized IPC at nRH = 1000
print(1-stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 1000)]['ramulator.normalized_ipc'].min())
# ABACuS average normalized memory latency at nRH = 1000
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 1000)]['ramulator.normalized_read_latency'].mean())

# ABACuS average normalized IPC at nRH = 125
print(1-stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125)]['ramulator.normalized_ipc'].mean())
# ABACuS min. normalized IPC at nRH = 125
print(1-stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125)]['ramulator.normalized_ipc'].min())
# ABACuS average normalized memory latency at nRH = 125
print(stats[(stats['config'] == 'ABACuS') & (stats['nrh'] == 125)]['ramulator.normalized_read_latency'].mean())

#### Figures 9 and 17

In [ ]:
MY_MECHANISM_NAME = 'ABACuS'

### READ RESULTS INTO PANDAS DATAFRAME
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

resultsdir = "../results"
# list all directories in resultsdir
configs = [d for d in os.listdir(resultsdir) if os.path.isdir(os.path.join(resultsdir, d))]
configs = ['Baseline.yaml', 'Hydra-Baseline.yaml', 'REGA125.yaml', 'REGA250.yaml', 'REGA500.yaml', 'REGA1000.yaml', 'Graphene125.yaml', 'Graphene250.yaml', 'Graphene500.yaml', 'Graphene1000.yaml', 'Hydra125.yaml', 'Hydra250.yaml', 'Hydra500.yaml', 'Hydra1000.yaml', 'PARA125.yaml', 'PARA250.yaml', 'PARA500.yaml', 'PARA1000.yaml', 'ABACUS125.yaml', 'ABACUS250.yaml', 'ABACUS500.yaml', 'ABACUS1000.yaml']

# print found configs
print('Found configs: {}'.format(configs))
# list all directories under all configs
workloads = []
for c in configs:
    workloads.append([d for d in os.listdir(os.path.join(resultsdir, c)) if os.path.isdir(os.path.join(resultsdir, c, d))])
# find only the intersection of all workloads
workloads = list(set.intersection(*map(set, workloads)))
# print found workloads
print('Found workloads: {}'.format(workloads))

stats_per_config_workload = []
workloads = [w for w in workloads if not '-' in w]

# for every config + workload directory
for c in configs:
    for w in workloads:
        # find all files in the directory
        files = [f for f in os.listdir(os.path.join(resultsdir, c, w)) if os.path.isfile(os.path.join(resultsdir, c, w, f))]
        # find the stats file
        stat_files = [f for f in files if f.endswith('output.txt')]
        # if there is a stats file
        if stat_files:
            for stat_file in stat_files:
                # if the stats_file has less than three lines skip it
                if len(open(os.path.join(resultsdir, c, w, stat_file)).readlines()) < 3:
                    continue
                
                # print the name of the stats_file
                print('Found stats file: {}'.format(os.path.join(os.path.join(resultsdir, c, w, stat_file))))

                lines = open(os.path.join(resultsdir, c, w, stat_file)).readlines()
                total_energy = 0
                for l in lines:
                    # if line contains nJ, add l.split()[-2] to total_energy
                    if 'Total Idle energy:' in l:
                        continue
                    if 'nJ' in l:
                        total_energy += float(l.split()[-2])
                    if l.startswith('REF CMD energy'):
                        break

                
                # create a df with the config, workload and total_energy
                df = pd.DataFrame({'config': [c], 'workload': [w], 'total_energy': [total_energy]})
                df.reset_index(inplace=True, drop=True)
                stats_per_config_workload.append(df)
        else:
            print('Config: {}, Workload: {}, Stats: No stats file found'.format(c, w))

# concatenate all stats into one dataframe
stats = pd.concat(stats_per_config_workload)

# find elements where workload does not contain '-'
# these are multi core workloads
stats = stats[~stats['workload'].str.contains('-')]

# remove these two workloads: stream_10.trace and random_10.trace
stats = stats[~stats['workload'].isin(['gups'])]
# also from workloads
workloads = [w for w in workloads if not w in ['gups']]

# remove "-16DR" from config names
stats['config'] = stats['config'].str.replace('-16DR', '')

# replace 1K with 1000 in config names
stats['config'] = stats['config'].str.replace('1K', '1000')

# replace 'Baseline' with 'Baseline0'
stats['config'] = stats['config'].str.replace('Baseline', 'Baseline0')

# add a new column that stores in integer the number in the config name
stats['nrh'] = stats['config'].str.extract('(\d+)').astype(int)

# remove numbers from config names
stats['config'] = stats['config'].str.replace('\d+', '', regex=True)

# remove yaml from config names
stats['config'] = stats['config'].str.replace('.yaml', '')

# replace SPR with APAR
stats['config'] = stats['config'].str.replace('ABACUS', MY_MECHANISM_NAME)

stats.loc[stats['workload'] == 'random_10.trace', 'workload'] = 'gups'

# increasing order of rbmpki
#order = ['511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', '500.perlbench', '523.xalancbmk', '510.parest', '557.xz', '482.sphinx3', '505.mcf', '436.cactusADM', '471.omnetpp', '473.astar', '483.xalancbmk', '462.libquantum', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf']
order = ['h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver', 'ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'stream_10.trace', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp']


# remove all workloads not in order
stats = stats[stats['workload'].isin(order)]
# also from the workload list
workloads = [w for w in workloads if w in order]


# order workloads according to the order
stats['workload'] = pd.Categorical(stats['workload'], categories=order, ordered=True)

stats_copy = stats.copy()


# use seaborn-deep style
sns.set(font_scale=1.0)
sns.set_style("whitegrid")
sns.set_palette("pastel", n_colors=1)

stats = stats_copy.copy()


# copy the IPC of the baseline config as to all configs
baseline = stats[stats['config'] == 'Baseline']
baseline = baseline[['workload', 'total_energy']]
# baseline
baseline.columns = ['workload', 'baseline_energy']
stats = pd.merge(stats, baseline, on='workload')

#hydra baseline
hydra_baseline = stats[stats['config'] == 'Hydra-Baseline']
hydra_baseline = hydra_baseline[['workload', 'total_energy']]
# hydra_baseline
hydra_baseline.columns = ['workload', 'ramulator.hydra_baseline_energy']
stats = pd.merge(stats, hydra_baseline, on='workload')

stats['normalized_energy'] = stats['total_energy'] / stats['baseline_energy']

# normalized ipc for hydra is not correct, so we overwrite it with the correct value
stats.loc[stats['config'].str.contains('Hydra'), 'normalized_energy'] = stats['total_energy'] / stats['ramulator.hydra_baseline_energy']

# add the geometric normalized ipc average as a new workload to every config
# geometric_mean = stats.groupby(['config','nrh'])['ramulator.normalized_ipc'].apply(lambda x: x.prod()**(1.0/len(x))).reset_index()
# geometric_mean['workload'] = 'GeoMean'

# stats = pd.concat([stats, geometric_mean])

# # order = ['GeoMean', '531.deepsjeng', '502.gcc', '541.leela', '435.gromacs', '481.wrf', '458.sjeng', '445.gobmk', '444.namd', '508.namd', '401.bzip2', '456.hmmer', '403.gcc', '464.h264ref', '526.blender', '447.dealII', '544.nab', '523.xalancbmk', '500.perlbench', '538.imagick', '525.x264', '507.cactuBSSN', '511.povray', '462.libquantum', '473.astar', '510.parest', '482.sphinx3', '505.mcf', '557.xz', '471.omnetpp', '483.xalancbmk', '436.cactusADM', '520.omnetpp', '450.soplex', '470.lbm', '519.lbm', '434.zeusmp', '433.milc', '459.GemsFDTD', '549.fotonik3d', '429.mcf', '437.leslie3d']
# order = ['GeoMean', 'h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver', 'ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp']

low_rbmpki = ['h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver']
med_rbmpki = ['ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2']
high_rbmpki = ['433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp']

# add new column called rbmpki_category, set category if workload matches one of the above lists
stats['rbmpki_category'] = np.where(stats['workload'].isin(low_rbmpki), 'low (<2)', np.where(stats['workload'].isin(med_rbmpki), 'medium (<10)', np.where(stats['workload'].isin(high_rbmpki), 'high (>10)', 'none')))

# geometric_mean = stats.groupby(['config','nrh','rbmpki_category'])['ramulator.normalized_ipc'].apply(lambda x: x.prod()**(1.0/len(x))).reset_index()
# geometric_mean['workload'] = 'GeoMean'
# stats = pd.concat([stats, geometric_mean])

# order and sort nRH high to low
stats['nrh'] = pd.Categorical(stats['nrh'], categories=[1000, 500, 250, 125], ordered=True)
stats = stats.sort_values('nrh')

# remove rows where rbmpki_category is none
stats = stats[stats['rbmpki_category'] != 'none']

order = ['low (<2)', 'medium (<10)', 'high (>10)']
stats['rbmpki_category'] = pd.Categorical(stats['rbmpki_category'], categories=order, ordered=True)

#barplot of normalized IPC, also draw edges around bars
fig, ax = plt.subplots(figsize=(7, 2))
ax = sns.boxplot(x='rbmpki_category', y='normalized_energy', hue='nrh', data=stats[(stats['config'] == 'ABACuS')], showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black"})

ax.set_xlabel('Row buffer misses per kilo instructions (RBMPKI)')
ax.set_ylabel('Normalized\nDRAM energy\ndistribution')
ax.axhline(y=1.0, color='#e74c3c', linestyle='--')
# write above the red line 'baseline IPC' using the same pastel red color
ax.text(0.01, 0.15, 'Baseline DRAM energy', color='#e74c3c', transform=ax.transAxes, fontsize=15)
# extend the y axis to 1.2
# ax.set_ylim(0.8, 1.05)
# color the 5th y tick red
ax.get_yticklabels()[1].set_color('#e74c3c')
# rotate x axis ticks
ax.set_xticklabels(ax.get_xticklabels())
# make axis tick font bigger
ax.tick_params(axis='both', which='major', labelsize=11)
ax.tick_params(axis='y', which='major', labelsize=12)

# have four y axis ticks
# ax.set_yticks([0.8, 0.85, 0.9, 0.95, 1.0, 1.05])

# # make x and y axis labels bigger
ax.xaxis.label.set_size(16)
# ax.yaxis.label.set_fontweight('bold')
ax.yaxis.label.set_size(16)
# ax.xaxis.label.set_fontweight('bold')
ax.add_patch(plt.Circle((2.303, 1.30), 0.05, color='blue', fill=False, clip_on=False))
ax.add_patch(plt.Circle((2.103, 1.71), 0.05, color='blue', fill=False, clip_on=False))
ax.add_patch(plt.Circle((1.903, 1.81), 0.05, color='blue', fill=False, clip_on=False))
ax.add_patch(plt.Circle((1.7, 2.01), 0.05, color='blue', fill=False, clip_on=False))
# write the minimum value of abacus at nrh = 1000
ax.text(1.55, 1.5, 'gups', color='blue', fontsize=12, bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.2', alpha = 0.95))

# # put the legend on top of the plot
# ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=4, fancybox=True, shadow=True, fontsize=10)
# # prepend "nRH" to legend names
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ['$N_{RH}$ = ' + label for label in labels], loc='upper center', bbox_to_anchor=(0.15, 1.0), ncol=1, fancybox=True, shadow=True, fontsize=10)

# remove legend
ax.legend_.remove()

ax.annotate('$N_{RH}=1000$ (leftmost box)\n$N_{RH}=500$\n$N_{RH}=250$\n$N_{RH}=125$ (rightmost box)', xy=(0.05, 0.90), xycoords='axes fraction',
            size=10, ha='left', va='top',
            bbox=dict(boxstyle='round', fc='w', ec='k'))



# plt.tight_layout()
plt.show()

# save figure
fig.savefig('abacus_energy_single_core_small.pdf', bbox_inches='tight')
# export data to csv
stats.to_csv('abacus_energy_single_core_small.csv', index=False)


# ABACuS average normalized energy at 1K nRH
print(stats[(stats['nrh'] == 1000) & (stats['config'] == 'ABACuS')]['normalized_energy'].mean())
# ABACuS max. normalized energy at 1K nRH
print(stats[(stats['nrh'] == 1000) & (stats['config'] == 'ABACuS')]['normalized_energy'].max())
# ABACuS average normalized energy at 125 nRH
print(stats[(stats['nrh'] == 125) & (stats['config'] == 'ABACuS')]['normalized_energy'].mean())
# ABACuS max. normalized energy at 125 nRH
print(stats[(stats['nrh'] == 125) & (stats['config'] == 'ABACuS')]['normalized_energy'].max())

# use seaborn-deep style
sns.set(font_scale=1.0)
sns.set_style("whitegrid")
sns.set_palette("pastel", n_colors=5)

stats = stats_copy.copy()

# copy the IPC of the baseline config as to all configs
baseline = stats[stats['config'] == 'Baseline']
baseline = baseline[['workload', 'total_energy']]
# baseline
baseline.columns = ['workload', 'baseline_energy']
stats = pd.merge(stats, baseline, on='workload')

#hydra baseline
hydra_baseline = stats[stats['config'] == 'Hydra-Baseline']
hydra_baseline = hydra_baseline[['workload', 'total_energy']]
# hydra_baseline
hydra_baseline.columns = ['workload', 'ramulator.hydra_baseline_energy']
stats = pd.merge(stats, hydra_baseline, on='workload')

stats['normalized_energy'] = stats['total_energy'] / stats['baseline_energy']

# normalized ipc for hydra is not correct, so we overwrite it with the correct value
stats.loc[stats['config'].str.contains('Hydra'), 'normalized_energy'] = stats['total_energy'] / stats['ramulator.hydra_baseline_energy']

# add the geometric normalized ipc average as a new workload to every config
geometric_mean = stats.groupby(['config','nrh'])['normalized_energy'].apply(lambda x: x.prod()**(1.0/len(x))).reset_index()
geometric_mean['workload'] = 'GeoMean'

stats = pd.concat([stats, geometric_mean])
# order in decreasing nRH (the nRH column) use pd.Categorical on nrh
stats['nrh'] = pd.Categorical(stats['nrh'], categories=[1000, 500, 250, 125], ordered=True)


#order = ['GeoMean', '531.deepsjeng', '502.gcc', '541.leela', '435.gromacs', '481.wrf', '458.sjeng', '445.gobmk', '444.namd', '508.namd', '401.bzip2', '456.hmmer', '403.gcc', '464.h264ref', '526.blender', '447.dealII', '544.nab', '523.xalancbmk', '500.perlbench', '538.imagick', '525.x264', '507.cactuBSSN', '511.povray', '462.libquantum', '473.astar', '510.parest', '482.sphinx3', '505.mcf', '557.xz', '471.omnetpp', '483.xalancbmk', '436.cactusADM', '520.omnetpp', '450.soplex', '470.lbm', '519.lbm', '434.zeusmp', '433.milc', '459.GemsFDTD', '549.fotonik3d', '429.mcf', '437.leslie3d']
order = ['h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver', 'ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp','GeoMean']

# order = ['GeoMean', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', '500.perlbench', '523.xalancbmk', '510.parest', '557.xz', '482.sphinx3', '505.mcf', '436.cactusADM', '471.omnetpp', '473.astar', '483.xalancbmk', '462.libquantum', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf']

stats['workload'] = pd.Categorical(stats['workload'], categories=order, ordered=True)
#barplot of normalized IPC, also draw edges around bars

fig, ax = plt.subplots(figsize=(13, 4))
ax = sns.barplot(x='workload', y='normalized_energy', hue='nrh', data=stats[(stats['config'] == 'ABACuS')], edgecolor='black', linewidth=0.5)

ax.set_xlabel('Workload')
ax.set_ylabel('Normalized DRAM Energy')
# move ylabel down
ax.yaxis.set_label_coords(-0.045,0.3)
# draw a red line at y = 1.0, label it as baseline IPC
ax.axhline(y=0.999, color='r', linestyle='--')
# write above the red line 'baseline IPC' using the same pastel red color, also draw a box around it
#ax.text(0.5, 0.99, 'baseline DRAM Energy', color='r', transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='r', boxstyle='round,pad=0.2'))
ax.text(0.01, 0.2, 'Baseline DRAM energy', color='#e74c3c', transform=ax.transAxes, fontsize=15, bbox=dict(facecolor='white', edgecolor='r', boxstyle='round,pad=0.2', alpha = 0.95))

# extend the y axis to 1.2
ax.set_ylim(0.90, 1.2)
# color the 5th y tick red
ax.get_yticklabels()[1].set_color('#e74c3c')
# rotate x axis ticks
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
# make axis tick font bigger
ax.tick_params(axis='both', which='major', labelsize=11)
ax.tick_params(axis='y', which='major', labelsize=12)

# # draw vertical lines to separate the rowhammer threshold values
ax.axvline(x=26.5, color='grey', linestyle='-', alpha=0.5)
# put text before the line saying "LOW RBMPKI"
ax.text(0.33, 0.2, 'LOW RBMPKI', color='grey', transform=ax.transAxes, fontsize=12, bbox=dict(facecolor='white', edgecolor='grey', boxstyle='round,pad=0.2', alpha = 0.95))
# put arrow to the left above text
ax.annotate('', xy=(19.5, 0.93), xytext=(25.5, 0.93), arrowprops=dict(facecolor='grey', shrink=0.01, width=3, headwidth=10, alpha=0.99))
ax.axvline(x=46.5, color='grey', linestyle='-', alpha=0.5)
ax.text(0.65, 0.2, 'MED. RBMPKI', color='grey', transform=ax.transAxes, fontsize=12, bbox=dict(facecolor='white', edgecolor='grey', boxstyle='round,pad=0.2', alpha = 0.99))
ax.annotate('', xy=(39.5, 0.93), xytext=(45.5, 0.93), arrowprops=dict(facecolor='grey', shrink=0.01, width=3, headwidth=10, alpha=0.95))
ax.axvline(x=61.5, color='grey', linestyle='-', alpha=0.5)
ax.text(0.875, 0.2, 'HIGH RBMPKI', color='grey', transform=ax.transAxes, fontsize=12, bbox=dict(facecolor='white', edgecolor='grey', boxstyle='round,pad=0.2', alpha = 0.99))
ax.annotate('', xy=(54.5, 0.93), xytext=(60.5, 0.93), arrowprops=dict(facecolor='grey', shrink=0.01, width=3, headwidth=10, alpha=0.95))

# make x and y axis labels bigger
ax.xaxis.label.set_size(16)
#ax.yaxis.label.set_fontweight('bold')
ax.yaxis.label.set_size(16)
#ax.xaxis.label.set_fontweight('bold')

# put the legend on top of the plot
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=4, fancybox=True, shadow=True, fontsize=12)
# prepend "nRH" to legend names
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ['$N_{RH}$ = ' + label for label in labels], loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=4, fancybox=True, shadow=True, fontsize=12)

# highlight the geometric mean ax label
ax.get_xticklabels()[62].set_fontweight('bold')

# draw a small orange arrow to the middle of the plot
ax.annotate('', xy=(54.5, 1.18), xytext=(56.5, 1.18), arrowprops=dict(facecolor='blue', shrink=0.01, width=3, headwidth=10, alpha=0.99))
# put text above the arrow
ax.text(0.835, 0.9, '2.01', color='blue', transform=ax.transAxes, fontsize=12, bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.2', alpha = 0.99))

# draw a small orange arrow to the middle of the plot
ax.annotate('', xy=(54.5, 1.11), xytext=(56.8, 1.15), arrowprops=dict(facecolor='#e67e22', shrink=0.01, width=3, headwidth=10, alpha=0.99))
# put text above the arrow
ax.text(0.835, 0.6, '1.81', color='orange', transform=ax.transAxes, fontsize=12, bbox=dict(facecolor='white', edgecolor='orange', boxstyle='round,pad=0.2', alpha = 0.99))

# draw a small orange arrow to the middle of the plot
ax.annotate('', xy=(58.5, 1.115), xytext=(57.0, 1.15), arrowprops=dict(facecolor='green', shrink=0.01, width=3, headwidth=10, alpha=0.99))
# put text above the arrow
ax.text(0.935, 0.6, '1.71', color='green', transform=ax.transAxes, fontsize=12, bbox=dict(facecolor='white', edgecolor='green', boxstyle='round,pad=0.2', alpha = 0.99))

# draw a small orange arrow to the middle of the plot
ax.annotate('', xy=(59.5, 1.18), xytext=(57.3, 1.18), arrowprops=dict(facecolor='red', shrink=0.01, width=3, headwidth=10, alpha=0.99))
# put text above the arrow
ax.text(0.955, 0.9, '1.30', color='red', transform=ax.transAxes, fontsize=12, bbox=dict(facecolor='white', edgecolor='red', boxstyle='round,pad=0.2', alpha = 0.99))


plt.tight_layout()
plt.show()

# save figure
fig.savefig('abacus_energy_single_core.pdf', bbox_inches='tight')
# export data to csv
stats.to_csv('abacus_energy_single_core.csv', index=False)



# numbers to put in paper

# ABACuS at 1000 nRH normalized_energy for the geomean workload
print(stats[(stats['workload'] == 'GeoMean') & (stats['nrh'] == 1000) & (stats['config'] == 'ABACuS')]['normalized_energy'])
# ABACuS at 1000 nRH maximum normalized_energy
print(stats[(stats['nrh'] == 1000) & (stats['config'] == 'ABACuS')]['normalized_energy'].max())

# the above for nrh = 125
print(stats[(stats['workload'] == 'GeoMean') & (stats['nrh'] == 125) & (stats['config'] == 'ABACuS')]['normalized_energy'])
print(stats[(stats['nrh'] == 125) & (stats['config'] == 'ABACuS')]['normalized_energy'].max())


evaluation_paragraph = """
ABACuS average normalized energy at 1K nRH {avg_energy_1k}
ABACuS max normalized energy at 1K nRH {max_energy_1k}

ABACuS average normalized energy at 125 nRH {avg_energy_125}
ABACuS max normalized energy at 125 nRH {max_energy_125}
""".format(
    avg_energy_1k=stats[(stats['nrh'] == 1000) & (stats['config'] == 'ABACuS')]['normalized_energy'].mean(),
    max_energy_1k=stats[(stats['nrh'] == 1000) & (stats['config'] == 'ABACuS')]['normalized_energy'].max(),
    avg_energy_125=stats[(stats['nrh'] == 125) & (stats['config'] == 'ABACuS')]['normalized_energy'].mean(),
    max_energy_125=stats[(stats['nrh'] == 125) & (stats['config'] == 'ABACuS')]['normalized_energy'].max(),
)

print(evaluation_paragraph)

### Performance comparison (single-core)

In [ ]:
MY_MECHANISM_NAME = 'ABACuS'

### READ RESULTS INTO PANDAS DATAFRAME
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

resultsdir = "../results"
# list all directories in resultsdir
configs = [d for d in os.listdir(resultsdir) if os.path.isdir(os.path.join(resultsdir, d))]
configs = ['Baseline.yaml', 'Hydra-Baseline.yaml', 'REGA125.yaml', 'REGA250.yaml', 'REGA500.yaml', 'REGA1000.yaml', 'Graphene125.yaml', 'Graphene250.yaml', 'Graphene500.yaml', 'Graphene1000.yaml', 'Hydra125.yaml', 'Hydra250.yaml', 'Hydra500.yaml', 'Hydra1000.yaml', 'PARA125.yaml', 'PARA250.yaml', 'PARA500.yaml', 'PARA1000.yaml', 
           'ABACUS125.yaml', 'ABACUS250.yaml', 'ABACUS500.yaml', 'ABACUS1000.yaml', 'ABACUS125-Big.yaml', 'ABACUS250-Big.yaml', 'ABACUS500-Big.yaml', 'ABACUS1000-Big.yaml']
# print found configs
print('Found configs: {}'.format(configs))
# list all directories under all configs
workloads = []
for c in configs:
    workloads.append([d for d in os.listdir(os.path.join(resultsdir, c)) if os.path.isdir(os.path.join(resultsdir, c, d))])
# find only the intersection of all workloads
workloads = list(set.intersection(*map(set, workloads)))
# print found workloads
print('Found workloads: {}'.format(workloads))

stats_per_config_workload = []
workloads = [w for w in workloads if not '-' in w]

# for every config + workload directory
for c in configs:
    for w in workloads:
        # find all files in the directory
        files = [f for f in os.listdir(os.path.join(resultsdir, c, w)) if os.path.isfile(os.path.join(resultsdir, c, w, f))]
        # find the stats file
        stat_files = [f for f in files if f.endswith('.stats')]
        # if there is a stats file
        if stat_files:
            for stat_file in stat_files:
                # if the stats_file has less than three lines skip it
                if len(open(os.path.join(resultsdir, c, w, stat_file)).readlines()) < 3:
                    continue
                
                # print the name of the stats_file
                print('Found stats file: {}'.format(os.path.join(os.path.join(resultsdir, c, w, stat_file))))

                extension = ''
                # if stats_file file name itself does not start with DDR4, parse it a bit
                if not stat_file.startswith('DDR4'):
                    # get the config name from the stats_file name
                    extension = '_'.join(stat_file.split('_')[:-1])
                    # prepend underscore to extension
                    extension = '_' + extension

                # read the stats file, name columns: 'name', 'value', 'description'
                df = pd.read_csv(os.path.join(resultsdir, c, w, stat_file), header=None).T
                df.columns = df.iloc[0]
                df.drop(0,inplace=True)
                # add a new column called 'config' with the config name
                df['config'] = c + extension
                # add a new column called 'workload' with the workload name
                df['workload'] = w
                # print the stats file
                # print('Config: {}, Workload: {}, Stats: {}'.format(c, w, df))
                # append the stats to the list
                df.reset_index(inplace=True, drop=True)
                stats_per_config_workload.append(df)
        else:
            print('Config: {}, Workload: {}, Stats: No stats file found'.format(c, w))

# concatenate all stats into one dataframe
stats = pd.concat(stats_per_config_workload)

# find elements where workload does not contain '-'
# these are multi core workloads
stats = stats[~stats['workload'].str.contains('-')]

# remove these two workloads: stream_10.trace and random_10.trace
stats = stats[~stats['workload'].isin(['gups'])]
# also from workloads
workloads = [w for w in workloads if not w in ['gups']]

# make sure config is string
stats['config'] = stats['config'].astype(str)

# remove "-16DR" from config names
stats['config'] = stats['config'].str.replace('-16DR', '')

# replace 1K with 1000 in config names
stats['config'] = stats['config'].str.replace('1K', '1000')

# replace 'Baseline' with 'Baseline0'
stats['config'] = stats['config'].str.replace('Baseline', 'Baseline0')

# add a new column that stores in integer the number in the config name
stats['nrh'] = stats['config'].str.extract('(\d+)').astype(int)

# remove numbers from config names
stats['config'] = stats['config'].str.replace('\d+', '', regex=True)

# remove yaml from config names
stats['config'] = stats['config'].str.replace('.yaml', '')

# replace SPR with APAR
stats['config'] = stats['config'].str.replace('ABACUS', MY_MECHANISM_NAME)

stats.loc[stats['workload'] == 'random_10.trace', 'workload'] = 'gups'

# increasing order of rbmpki
# order = ['511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', '500.perlbench', '523.xalancbmk', '510.parest', '557.xz', '482.sphinx3', '505.mcf', '436.cactusADM', '471.omnetpp', '473.astar', '483.xalancbmk', '462.libquantum', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf']
order = ['h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver', 'ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'stream_10.trace', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp']

# remove all workloads not in order
stats = stats[stats['workload'].isin(order)]
# also from the workload list
workloads = [w for w in workloads if w in order]

# order workloads according to the order
stats['workload'] = pd.Categorical(stats['workload'], categories=order, ordered=True)

stats_copy = stats.copy()

# import MultipleLocator
from matplotlib.ticker import MultipleLocator

# use seaborn-deep style
sns.set(font_scale=1.0)
sns.set_style("whitegrid")
sns.set_palette("pastel", n_colors=5)

stats = stats_copy.copy()
# instructions per cycle (IPC) is record_cycles_insts_0 / record_cycs_core_0
stats['ramulator.ipc'] = stats['ramulator.record_insts_core_0'] / stats['ramulator.record_cycs_core_0']


stats['ramulator.rbmpki'] = (stats['ramulator.row_conflicts_channel_0_core'] + stats['ramulator.row_misses_channel_0_core']) /\
                            stats['ramulator.record_insts_core_0'] * 1000

# copy the IPC of the baseline config as to all configs
baseline = stats[stats['config'] == 'Baseline']
baseline = baseline[['workload', 'ramulator.ipc', 'ramulator.read_latency_avg_0', 'ramulator.rbmpki', 'ramulator.window_full_stall_cycles_core_0']]
# baseline
baseline.columns = ['workload', 'ramulator.baseline_ipc', 'ramulator.baseline_read_latency_avg_0', 'ramulator.baseline_rbmpki', 'ramulator.baseline_stall_cycles']
stats = pd.merge(stats, baseline, on='workload')
#hydra baseline
hydra_baseline = stats[stats['config'] == 'Hydra-Baseline']
hydra_baseline = hydra_baseline[['workload', 'ramulator.ipc']]
# hydra_baseline
hydra_baseline.columns = ['workload', 'ramulator.hydra_baseline_ipc']
stats = pd.merge(stats, hydra_baseline, on='workload')

stats['ramulator.normalized_ipc'] = stats['ramulator.ipc'] / stats['ramulator.baseline_ipc']
stats['ramulator.normalized_read_latency'] = stats['ramulator.read_latency_avg_0'] / stats['ramulator.baseline_read_latency_avg_0']
stats['ramulator.normalized_stall_cycles'] = stats['ramulator.window_full_stall_cycles_core_0'] / stats['ramulator.baseline_stall_cycles']
stats['ramulator.normalized_rbmpki'] = stats['ramulator.rbmpki'] / stats['ramulator.baseline_rbmpki']

# # instructions per cycle (IPC) is record_cycles_insts_0 / record_cycs_core_0
# stats['ramulator.ipc'] = stats['ramulator.record_insts_core_0'] / stats['ramulator.record_cycs_core_0']

# # copy the IPC of the baseline config as to all configs
# baseline = stats[stats['config'] == 'Baseline']
# baseline = baseline[['workload', 'ramulator.ipc']]
# # baseline
# baseline.columns = ['workload', 'ramulator.baseline_ipc']
# stats = pd.merge(stats, baseline, on='workload')

# #hydra baseline
# hydra_baseline = stats[stats['config'] == 'Hydra-Baseline']
# hydra_baseline = hydra_baseline[['workload', 'ramulator.ipc']]
# # hydra_baseline
# hydra_baseline.columns = ['workload', 'ramulator.hydra_baseline_ipc']
# stats = pd.merge(stats, hydra_baseline, on='workload')

# stats['ramulator.normalized_ipc'] = stats['ramulator.ipc'] / stats['ramulator.baseline_ipc']

# normalized ipc for hydra is not correct, so we overwrite it with the correct value
stats.loc[stats['config'].str.contains('Hydra'), 'ramulator.normalized_ipc'] = stats['ramulator.ipc'] / stats['ramulator.hydra_baseline_ipc']


# new dataframe that does not have the baseline configs
stats_no_baseline = stats[~stats['config'].str.contains('Baseline')]

# new dataframe that does not have the baseline configs
print(stats_no_baseline['config'].unique())

# order nRH from high to low
stats_no_baseline['nrh'] = pd.Categorical(stats_no_baseline['nrh'], categories=[1000, 500, 250, 125], ordered=True)

# order config in this order: abacus, Graphene, Hydra, REGA, PARA
stats_no_baseline['config'] = pd.Categorical(stats_no_baseline['config'], categories=['ABACuS', 'Graphene', 'Hydra', 'REGA', 'PARA'], ordered=True)

#boxplot of normalized IPC
fig, ax = plt.subplots(figsize=(10, 4))
# show mean values as well
ax = sns.boxplot(x="nrh", y="ramulator.normalized_ipc", hue="config", data=stats_no_baseline, showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black"})
ax.set_xlabel('RowHammer Threshold ($N_{RH}$)')
ax.set_ylabel('Normalized IPC Distribution')
# draw a red line at y = 1.0, label it as baseline IPC
ax.axhline(y=1.0, color='r', linestyle='--')
# write above the red line 'baseline IPC'
ax.text(0.02, 0.92, 'Baseline IPC', color='#e74c3c', transform=ax.transAxes, fontsize=15)
# extend the y axis to 1.2
ax.set_ylim(0.2, 1.1)
# y axis tick every 0.1 increment
ax.yaxis.set_major_locator(MultipleLocator(0.1))
# color the 5th y tick red
ax.get_yticklabels()[9].set_color('#e74c3c')
# make axis tick font bigger
ax.tick_params(axis='both', which='major', labelsize=14)
# draw vertical lines to separate the rowhammer threshold values
ax.axvline(x=0.5, color='grey', linestyle='-', alpha=0.5)
ax.axvline(x=1.5, color='grey', linestyle='-', alpha=0.5)
ax.axvline(x=2.5, color='grey', linestyle='-', alpha=0.5)
# make x and y axis labels bigger
ax.xaxis.label.set_size(16)
ax.yaxis.label.set_size(16)

# put the legend on top of the plot
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.2), ncol=5, fancybox=True, shadow=True, fontsize=12)

plt.tight_layout()
plt.show()

# save figure
fig.savefig('abacus_performance_comparison_single_core.pdf', bbox_inches='tight')

# list mean normalized_ipc at 1000 nRH for all configs
# print(stats_no_baseline.groupby(['config','nrh'])['ramulator.normalized_ipc'].mean())

# Average normalized IPC for REGA at nRH = 125
print(1-stats_no_baseline[(stats_no_baseline['config'] == 'REGA') & (stats_no_baseline['nrh'] == 125)]['ramulator.normalized_ipc'].mean())
# Same for PARA at 1000 and 125
print(1-stats_no_baseline[(stats_no_baseline['config'] == 'PARA') & (stats_no_baseline['nrh'] == 1000)]['ramulator.normalized_ipc'].mean())
print(1-stats_no_baseline[(stats_no_baseline['config'] == 'PARA') & (stats_no_baseline['nrh'] == 125)]['ramulator.normalized_ipc'].mean())
# Average normalized IPC for ABACuS divided by Hydra at each nRH
print(1/stats_no_baseline[(stats_no_baseline['config'] == 'Hydra') & (stats_no_baseline['nrh'] == 1000)]['ramulator.normalized_ipc'].mean()/stats_no_baseline[(stats_no_baseline['config'] == 'ABACuS') & (stats_no_baseline['nrh'] == 1000)]['ramulator.normalized_ipc'].mean())
print(1/stats_no_baseline[(stats_no_baseline['config'] == 'Hydra') & (stats_no_baseline['nrh'] == 500)]['ramulator.normalized_ipc'].mean()/stats_no_baseline[(stats_no_baseline['config'] == 'ABACuS') & (stats_no_baseline['nrh'] == 500)]['ramulator.normalized_ipc'].mean())
print(1/stats_no_baseline[(stats_no_baseline['config'] == 'Hydra') & (stats_no_baseline['nrh'] == 250)]['ramulator.normalized_ipc'].mean()/stats_no_baseline[(stats_no_baseline['config'] == 'ABACuS') & (stats_no_baseline['nrh'] == 250)]['ramulator.normalized_ipc'].mean())
print(1/stats_no_baseline[(stats_no_baseline['config'] == 'Hydra') & (stats_no_baseline['nrh'] == 125)]['ramulator.normalized_ipc'].mean()/stats_no_baseline[(stats_no_baseline['config'] == 'ABACuS') & (stats_no_baseline['nrh'] == 125)]['ramulator.normalized_ipc'].mean())

# average rbmpki of Hydra at nRH 125 divided by ABACuS at nRH 125
print(1/(stats_no_baseline[(stats_no_baseline['config'] == 'ABACuS') & (stats_no_baseline['nrh'] == 125)]['ramulator.normalized_rbmpki'].mean()/stats_no_baseline[(stats_no_baseline['config'] == 'Hydra') & (stats_no_baseline['nrh'] == 125)]['ramulator.normalized_rbmpki'].mean()))
# same for memory latency
print(1/(stats_no_baseline[(stats_no_baseline['config'] == 'ABACuS') & (stats_no_baseline['nrh'] == 125)]['ramulator.normalized_read_latency'].mean()/stats_no_baseline[(stats_no_baseline['config'] == 'Hydra') & (stats_no_baseline['nrh'] == 125)]['ramulator.normalized_read_latency'].mean()))

# mean number of preventive refreshes for ABACuS at 125 nrh divided by graphene
print(stats_no_baseline[(stats_no_baseline['config'] == 'ABACuS') & (stats_no_baseline['nrh'] == 125)]['ramulator.preventive_refreshes_channel_0_core'].mean()/stats_no_baseline[(stats_no_baseline['config'] == 'Graphene') & (stats_no_baseline['nrh'] == 125)]['ramulator.preventive_refreshes_channel_0_core'].mean())
# mean number of stall cycles for the same
print(1/(stats_no_baseline[(stats_no_baseline['config'] == 'ABACuS') & (stats_no_baseline['nrh'] == 125)]['ramulator.normalized_stall_cycles'].mean()/stats_no_baseline[(stats_no_baseline['config'] == 'Graphene') & (stats_no_baseline['nrh'] == 125)]['ramulator.normalized_stall_cycles'].mean()))


#### Performance comparison (8-cores)

In [ ]:
MY_MECHANISM_NAME = 'ABACuS'

### READ RESULTS INTO PANDAS DATAFRAME
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

resultsdir = "../results"
# list all directories in resultsdir
configs = [d for d in os.listdir(resultsdir) if os.path.isdir(os.path.join(resultsdir, d))]
configs = ['Baseline.yaml', 'MC-Baseline.yaml', 'Hydra-Baseline.yaml', 'REGA125.yaml', 'REGA250.yaml', 'REGA500.yaml', 'REGA1000.yaml', 'Graphene125.yaml', 'Graphene250.yaml', 'Graphene500.yaml', 'Graphene1000.yaml', 'Hydra125.yaml', 'Hydra250.yaml', 'Hydra500.yaml', 'Hydra1000.yaml', 'PARA125.yaml', 'PARA250.yaml', 'PARA500.yaml', 'PARA1000.yaml', 
           'ABACUS125.yaml', 'ABACUS250.yaml', 'ABACUS500.yaml', 'ABACUS1000.yaml', 'ABACUS125-Big.yaml', 'ABACUS250-Big.yaml', 'ABACUS500-Big.yaml', 'ABACUS1000-Big.yaml']
# print found configs
print('Found configs: {}'.format(configs))
# list all directories under all configs
workloads = []
for c in configs:
    workloads.append([d for d in os.listdir(os.path.join(resultsdir, c)) if os.path.isdir(os.path.join(resultsdir, c, d))])
# find only the intersection of all workloads
# workloads = list(set.intersection(*map(set, workloads)))
# keep only unique workloads
workloads = list(set([item for sublist in workloads for item in sublist]))

# print found workloads
print('Found workloads: {}'.format(workloads))

stats_per_config_workload = []

# for every config + workload directory
for c in configs:
    for w in workloads:
        # find all files in the directory
        if os.path.isdir(os.path.join(resultsdir, c, w)):
            files = [f for f in os.listdir(os.path.join(resultsdir, c, w)) if os.path.isfile(os.path.join(resultsdir, c, w, f))]
            # find the stats file
            stat_files = [f for f in files if f.endswith('.stats')]
            # if there is a stats file
            if stat_files:
                for stat_file in stat_files:
                    # if the stats_file has less than three lines skip it
                    if len(open(os.path.join(resultsdir, c, w, stat_file)).readlines()) < 3:
                        continue
                    
                    # print the name of the stats_file
                    print('Found stats file: {}'.format(os.path.join(os.path.join(resultsdir, c, w, stat_file))))

                    extension = ''
                    # if stats_file file name itself does not start with DDR4, parse it a bit
                    if not stat_file.startswith('DDR4'):
                        # get the config name from the stats_file name
                        extension = '_'.join(stat_file.split('_')[:-1])
                        # prepend underscore to extension
                        extension = '_' + extension

                    # read the stats file, name columns: 'name', 'value', 'description'
                    df = pd.read_csv(os.path.join(resultsdir, c, w, stat_file), header=None).T
                    df.columns = df.iloc[0]
                    df.drop(0,inplace=True)
                    # add a new column called 'config' with the config name
                    df['config'] = c + extension
                    # add a new column called 'workload' with the workload name
                    df['workload'] = w
                    # print the stats file
                    # print('Config: {}, Workload: {}, Stats: {}'.format(c, w, df))
                    # append the stats to the list
                    df.reset_index(inplace=True, drop=True)
                    stats_per_config_workload.append(df)
            else:
                print('Config: {}, Workload: {}, Stats: No stats file found'.format(c, w))

# concatenate all stats into one dataframe
stats = pd.concat(stats_per_config_workload)

# remove workloads that have "gups" in them
stats = stats[~stats['workload'].str.contains('gups')]

# grep_map0 produced 0 cycles for any core except 7, weird bug, ignore for now
# stats = stats[~stats['workload'].str.contains('grep_map0')]

# also remove them from the list
workloads = [w for w in workloads if 'gups' not in w]

# grep_map0 produced 0 cycles for any core except 7, weird bug, ignore for now
# workloads = [w for w in workloads if 'grep_map0' not in w]


# replace all instances of "random_10.trace" in workload names with "gups"
stats['workload'] = stats['workload'].str.replace('random_10.trace', 'gups')

# all workloads
HIGH_RBMPKI = ['h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver', 'ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'stream_10.trace', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp']

# HIGH_RBMPKI = ['519.lbm', '459.GemsFDTD', '450.soplex', 'h264_decode', '520.omnetpp', '433.milc', '434.zeusmp', 'bfs_dblp', '429.mcf', '549.fotonik3d', '470.lbm', 'bfs_ny', 'bfs_cm2003', '437.leslie3d', 'gups']

# HIGH_RBMPKI = ['h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver', 'ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'stream_10.trace', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'random_10.trace', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp']

#HIGH_RBMPKI = ['531.deepsjeng', '502.gcc', '541.leela', '435.gromacs', '481.wrf', '458.sjeng', '445.gobmk', '444.namd', '508.namd', '401.bzip2', '456.hmmer', '403.gcc', '464.h264ref', '526.blender', '447.dealII', '544.nab', '523.xalancbmk', '500.perlbench', '538.imagick', '525.x264', '507.cactuBSSN', '511.povray', '462.libquantum', '473.astar', '510.parest', '482.sphinx3', '505.mcf', '557.xz', '471.omnetpp', '483.xalancbmk', '436.cactusADM', '520.omnetpp', '450.soplex', '470.lbm', '519.lbm', '434.zeusmp', '433.milc', '459.GemsFDTD', '549.fotonik3d', '429.mcf', '437.leslie3d']
stats = stats[stats['workload'].str.contains('|'.join(HIGH_RBMPKI))]

# remove h264_decode fro workloads
# stats = stats[~stats['workload'].str.contains('h264_decode')]
# remove h264_decode from HIGH_RBMPKI
# HIGH_RBMPKI.remove('h264_decode')

# keep mc_only_stats only for workloads that contain the strings in HIGH_RBMPKI

# remove "-16DR" from config names
stats['config'] = stats['config'].str.replace('-16DR', '')

# replace 1K with 1000 in config names
stats['config'] = stats['config'].str.replace('1K', '1000')

# replace 'Baseline' with 'Baseline0'
stats['config'] = stats['config'].str.replace('Baseline', 'Baseline0')

# add a new column that stores in integer the number in the config name
stats['nrh'] = stats['config'].str.extract('(\d+)').astype(int)

# remove numbers from config names
stats['config'] = stats['config'].str.replace('\d+', '', regex=True)

# remove yaml from config names
stats['config'] = stats['config'].str.replace('.yaml', '')

# replace SPR with APAR
stats['config'] = stats['config'].str.replace('ABACUS', MY_MECHANISM_NAME)

stats_copy = stats.copy()

# remove from mc_only_stats the workloads that contain less than 7 dashes
mc_only_stats = stats[stats['workload'].str.count('-') >= 7]
# remove from mc_only_stats workloads that contain stream_10.trace or random_10.trace
mc_only_stats = mc_only_stats[~mc_only_stats['workload'].str.contains('stream_10.trace')]
mc_only_stats = mc_only_stats[~mc_only_stats['workload'].str.contains('random_10.trace')]
sc_only_stats = stats[~stats['workload'].str.contains('-')].copy()

# expand the workload column into four columns by splitting using '-'
mc_only_stats[['wl0', 'wl1', 'wl2', 'wl3', 'wl4', 'wl5', 'wl6', 'wl7']] = mc_only_stats['workload'].str.split('-', expand=True)
sc_only_stats['ramulator.ipc'] = sc_only_stats['ramulator.record_insts_core_0'] / sc_only_stats['ramulator.record_cycs_core_0']

# for each ramulator.record_insts_core_i column, if ramulator.record_cycs_core_i is 0, set it to 1 (This is to prevent division by zero)
for i in range(0, 8):
    mc_only_stats.loc[mc_only_stats['ramulator.record_cycs_core_{}'.format(i)] == 0, 'ramulator.record_cycs_core_{}'.format(i)] = 1

# for each ramulator.record_insts_core_i column, divide it by ramulator.record_cycs_core_i column and add it as ipci column
for i in range(0, 8):
    mc_only_stats['ipc{}'.format(i)] = mc_only_stats['ramulator.record_insts_core_{}'.format(i)] / mc_only_stats['ramulator.record_cycs_core_{}'.format(i)]

# write to csv mc_only_stats but only the config, workload and ipci columns
mc_only_stats[['config', 'wl0', 'wl1', 'wl2', 'wl3', 'wl4', 'wl5', 'wl6', 'wl7', 'ipc0', 'ipc1', 'ipc2', 'ipc3', 'ipc4', 'ipc5', 'ipc6', 'ipc7', 'nrh']].to_csv('mc_only_stats.csv', index=False)

# use seaborn-deep style
sns.set(font_scale=1.0)
sns.set_style("whitegrid")
sns.set_palette("pastel", n_colors=5)

stats_parsed = pd.read_csv('mc_only_stats.csv')

stats_parsed['workload'] = stats_parsed['wl0']

# copy the IPC of the baseline config as to all configs
baseline = sc_only_stats[sc_only_stats['config'] == 'MC-Baseline']
baseline = baseline[['workload', 'ramulator.ipc']]

# baseline
baseline.columns = ['workload', 'ramulator.baseline_ipc']
stats_parsed = pd.merge(stats_parsed, baseline, on='workload')

# for all ipc columns, divide by the baseline ipc
for i in range(0, 8):
    stats_parsed['normalized_ipc{}'.format(i)] = stats_parsed['ipc{}'.format(i)] / stats_parsed['ramulator.baseline_ipc']

stats_parsed['weighted_speedup'] = stats_parsed['normalized_ipc0'] + stats_parsed['normalized_ipc1'] + stats_parsed['normalized_ipc2'] + stats_parsed['normalized_ipc3'] + stats_parsed['normalized_ipc4'] + stats_parsed['normalized_ipc5'] + stats_parsed['normalized_ipc6'] + stats_parsed['normalized_ipc7']

baselinepp = stats_parsed[stats_parsed['config'] == 'Baseline']
baselinepp = baselinepp[['workload', 'weighted_speedup']]
# baseline
baselinepp.columns = ['workload', 'baseline_weighted_speedup']
stats_parsed = pd.merge(stats_parsed, baselinepp, on='workload')

stats_parsed['normalized_weighted_speedup'] = stats_parsed['weighted_speedup'] / stats_parsed['baseline_weighted_speedup']

# for normalized_weighted_speedups above 1.0, make them 1.0
# stats_parsed.loc[stats_parsed['normalized_weighted_speedup'] > 1.0, 'normalized_weighted_speedup'] = 1.0

stats_parsed = stats_parsed[~stats_parsed['config'].str.contains('Baseline')]

# print(stats_parsed['workload'].unique())

# order nRH from high to low
stats_parsed['nrh'] = pd.Categorical(stats_parsed['nrh'], categories=[1000, 500, 250, 125], ordered=True)

# order config in this order: abacus, Graphene, Hydra, REGA, PARA
stats_parsed['config'] = pd.Categorical(stats_parsed['config'], categories=['ABACuS', 'Graphene', 'Hydra', 'REGA', 'PARA'], ordered=True)

#boxplot of normalized IPC
fig, ax = plt.subplots(figsize=(10, 4))

# show mean values do not plot outliers
ax = sns.boxplot(x="nrh", y="normalized_weighted_speedup", hue="config", data=stats_parsed, showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black"})#, showfliers=False)

ax.set_xlabel('RowHammer Threshold ($N_{RH}$)')
ax.set_ylabel('Normalized Weighted Speedup')
# draw a red line at y = 1.0, label it as baseline IPC
ax.axhline(y=1.0, color='r', linestyle='--')
# write above the red line 'baseline IPC'
ax.text(0.02, 0.94, 'Baseline weighted speedup', color='#e74c3c', transform=ax.transAxes, fontsize=15)
# extend the y axis to 1.2
ax.set_ylim(0, 1.1)
# color the 5th y tick red
ax.get_yticklabels()[5].set_color('#e74c3c')
# make axis tick font bigger
ax.tick_params(axis='both', which='major', labelsize=14)
# draw vertical lines to separate the rowhammer threshold values
ax.axvline(x=0.5, color='grey', linestyle='-', alpha=0.5)
ax.axvline(x=1.5, color='grey', linestyle='-', alpha=0.5)
ax.axvline(x=2.5, color='grey', linestyle='-', alpha=0.5)
# make x and y axis labels bigger
ax.xaxis.label.set_size(16)
ax.yaxis.label.set_size(16)

# put the legend on top of the plot
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.2), ncol=5, fancybox=True, shadow=True, fontsize=12)

plt.tight_layout()
plt.show()

# save figure
fig.savefig('abacus_performance_comparison_multi_core.pdf', bbox_inches='tight')
# export csv to file
stats_parsed.to_csv('abacus_performance_comparison_multi_core.csv', index=False)


# average normalized ws across all workloads for ABACuS at 1K nRH (print in one line)
print(1-(stats_parsed[(stats_parsed['config'] == 'ABACuS') & (stats_parsed['nrh'] == 1000)]['normalized_weighted_speedup'].mean()))
# same for other 3 thresholds
print(1-(stats_parsed[(stats_parsed['config'] == 'ABACuS') & (stats_parsed['nrh'] == 500)]['normalized_weighted_speedup'].mean()))
print(1-(stats_parsed[(stats_parsed['config'] == 'ABACuS') & (stats_parsed['nrh'] == 250)]['normalized_weighted_speedup'].mean()))
print(1-(stats_parsed[(stats_parsed['config'] == 'ABACuS') & (stats_parsed['nrh'] == 125)]['normalized_weighted_speedup'].mean()))

#Hydra's average normalized ws across all workloads at 1K nRH (print in one line) divided by ABACuS's
print(-(stats_parsed[(stats_parsed['config'] == 'Hydra') & (stats_parsed['nrh'] == 1000)]['normalized_weighted_speedup'].mean())+(stats_parsed[(stats_parsed['config'] == 'ABACuS') & (stats_parsed['nrh'] == 1000)]['normalized_weighted_speedup'].mean()))

print(1/(stats_parsed[(stats_parsed['config'] == 'ABACuS') & (stats_parsed['nrh'] == 125)]['normalized_weighted_speedup'].mean()))
# Just Hydra's overhead at nRH 125
print(1-(stats_parsed[(stats_parsed['config'] == 'Hydra') & (stats_parsed['nrh'] == 125)]['normalized_weighted_speedup'].mean()))
# REGA's
print(1-(stats_parsed[(stats_parsed['config'] == 'REGA') & (stats_parsed['nrh'] == 125)]['normalized_weighted_speedup'].mean()))
# PARA's
print(1-(stats_parsed[(stats_parsed['config'] == 'PARA') & (stats_parsed['nrh'] == 125)]['normalized_weighted_speedup'].mean()))

# Graphene's at nRH 1000
print(1-(stats_parsed[(stats_parsed['config'] == 'Graphene') & (stats_parsed['nrh'] == 1000)]['normalized_weighted_speedup'].mean()))

print(1-(stats_parsed[(stats_parsed['config'] == 'ABACuS') & (stats_parsed['nrh'] == 1000)]['normalized_weighted_speedup'].min()))

# Hydra max overhead at 125
print(1-(stats_parsed[(stats_parsed['config'] == 'Hydra') & (stats_parsed['nrh'] == 125)]['normalized_weighted_speedup'].min()))


#### Energy comparison (single-core)

In [ ]:
MY_MECHANISM_NAME = 'ABACuS'

### READ RESULTS INTO PANDAS DATAFRAME
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

resultsdir = "../results"
# list all directories in resultsdir
configs = [d for d in os.listdir(resultsdir) if os.path.isdir(os.path.join(resultsdir, d))]
configs = ['Baseline.yaml', 'Hydra-Baseline.yaml', 'REGA125.yaml', 'REGA250.yaml', 'REGA500.yaml', 'REGA1000.yaml', 'Graphene125.yaml', 'Graphene250.yaml', 'Graphene500.yaml', 'Graphene1000.yaml', 'Hydra125.yaml', 'Hydra250.yaml', 'Hydra500.yaml', 'Hydra1000.yaml', 'PARA125.yaml', 'PARA250.yaml', 'PARA500.yaml', 'PARA1000.yaml', 'ABACUS125.yaml', 'ABACUS250.yaml', 'ABACUS500.yaml', 'ABACUS1000.yaml']

# print found configs
print('Found configs: {}'.format(configs))
# list all directories under all configs
workloads = []
for c in configs:
    workloads.append([d for d in os.listdir(os.path.join(resultsdir, c)) if os.path.isdir(os.path.join(resultsdir, c, d))])
# find only the intersection of all workloads
workloads = list(set.intersection(*map(set, workloads)))
# print found workloads
print('Found workloads: {}'.format(workloads))

stats_per_config_workload = []
workloads = [w for w in workloads if not '-' in w]

# for every config + workload directory
for c in configs:
    for w in workloads:
        # find all files in the directory
        files = [f for f in os.listdir(os.path.join(resultsdir, c, w)) if os.path.isfile(os.path.join(resultsdir, c, w, f))]
        # find the stats file
        stat_files = [f for f in files if f.endswith('output.txt')]
        # if there is a stats file
        if stat_files:
            for stat_file in stat_files:
                # if the stats_file has less than three lines skip it
                if len(open(os.path.join(resultsdir, c, w, stat_file)).readlines()) < 3:
                    continue
                
                # print the name of the stats_file
                print('Found stats file: {}'.format(os.path.join(os.path.join(resultsdir, c, w, stat_file))))

                lines = open(os.path.join(resultsdir, c, w, stat_file)).readlines()
                total_energy = 0
                for l in lines:
                    # if line contains nJ, add l.split()[-2] to total_energy
                    if 'Total Idle energy:' in l:
                        continue
                    if 'nJ' in l:
                        total_energy += float(l.split()[-2])
                    if l.startswith('REF CMD energy'):
                        break

                
                # create a df with the config, workload and total_energy
                df = pd.DataFrame({'config': [c], 'workload': [w], 'total_energy': [total_energy]})
                df.reset_index(inplace=True, drop=True)
                stats_per_config_workload.append(df)
        else:
            print('Config: {}, Workload: {}, Stats: No stats file found'.format(c, w))

# concatenate all stats into one dataframe
stats = pd.concat(stats_per_config_workload)

# find elements where workload does not contain '-'
# these are multi core workloads
stats = stats[~stats['workload'].str.contains('-')]

# remove these two workloads: stream_10.trace and random_10.trace
stats = stats[~stats['workload'].isin(['gups'])]
# also from workloads
workloads = [w for w in workloads if not w in ['gups']]

# remove "-16DR" from config names
stats['config'] = stats['config'].str.replace('-16DR', '')

# replace 1K with 1000 in config names
stats['config'] = stats['config'].str.replace('1K', '1000')

# replace 'Baseline' with 'Baseline0'
stats['config'] = stats['config'].str.replace('Baseline', 'Baseline0')

# add a new column that stores in integer the number in the config name
stats['nrh'] = stats['config'].str.extract('(\d+)').astype(int)

# remove numbers from config names
stats['config'] = stats['config'].str.replace('\d+', '', regex=True)

# remove yaml from config names
stats['config'] = stats['config'].str.replace('.yaml', '')

# replace SPR with APAR
stats['config'] = stats['config'].str.replace('ABACUS', MY_MECHANISM_NAME)

stats.loc[stats['workload'] == 'random_10.trace', 'workload'] = 'gups'

# increasing order of rbmpki
#order = ['511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', '500.perlbench', '523.xalancbmk', '510.parest', '557.xz', '482.sphinx3', '505.mcf', '436.cactusADM', '471.omnetpp', '473.astar', '483.xalancbmk', '462.libquantum', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf']
order = ['h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver', 'ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'stream_10.trace', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp']


# remove all workloads not in order
stats = stats[stats['workload'].isin(order)]
# also from the workload list
workloads = [w for w in workloads if w in order]


# order workloads according to the order
stats['workload'] = pd.Categorical(stats['workload'], categories=order, ordered=True)

stats_copy = stats.copy()


# use seaborn-deep style
sns.set(font_scale=1.0)
sns.set_style("whitegrid")
sns.set_palette("pastel", n_colors=5)

stats = stats_copy.copy()

# copy the IPC of the baseline config as to all configs
baseline = stats[stats['config'] == 'Baseline']
baseline = baseline[['workload', 'total_energy']]
# baseline
baseline.columns = ['workload', 'baseline_energy']
stats = pd.merge(stats, baseline, on='workload')

#hydra baseline
hydra_baseline = stats[stats['config'] == 'Hydra-Baseline']
hydra_baseline = hydra_baseline[['workload', 'total_energy']]
# hydra_baseline
hydra_baseline.columns = ['workload', 'ramulator.hydra_baseline_energy']
stats = pd.merge(stats, hydra_baseline, on='workload')

stats['normalized_energy'] = stats['total_energy'] / stats['baseline_energy']

# normalized ipc for hydra is not correct, so we overwrite it with the correct value
stats.loc[stats['config'].str.contains('Hydra'), 'normalized_energy'] = stats['total_energy'] / stats['ramulator.hydra_baseline_energy']

# new dataframe that does not have the baseline configs
stats_no_baseline = stats[~stats['config'].str.contains('Baseline')]

# new dataframe that does not have the baseline configs
print(stats_no_baseline['config'].unique())

# order nRH from high to low
stats_no_baseline['nrh'] = pd.Categorical(stats_no_baseline['nrh'], categories=[1000, 500, 250, 125], ordered=True)

# order config in this order: abacus, Graphene, Hydra, REGA, PARA
stats_no_baseline['config'] = pd.Categorical(stats_no_baseline['config'], categories=['ABACuS', 'Graphene', 'Hydra', 'REGA', 'PARA'], ordered=True)

#boxplot of normalized IPC
fig, ax = plt.subplots(figsize=(10, 4))
# show mean values as well
ax = sns.boxplot(x="nrh", y="normalized_energy", hue="config", data=stats_no_baseline, showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black"}, showfliers = True)
ax.set_xlabel('RowHammer Threshold ($N_{RH}$)')
ax.set_ylabel('Normalized Energy Distribution')
# draw a red line at y = 1.0, label it as baseline IPC
ax.axhline(y=1.0, color='r', linestyle='--')
# write above the red line 'baseline IPC'
ax.text(0.02, 0.02, 'Baseline DRAM energy', color='#e74c3c', transform=ax.transAxes, fontsize=15)
# extend the y axis to 1.2
ax.set_ylim(0.8, 2.5)
# color the 5th y tick red
ax.get_yticklabels()[1].set_color('#e74c3c')
# make axis tick font bigger
ax.tick_params(axis='both', which='major', labelsize=14)
# draw vertical lines to separate the rowhammer threshold values
ax.axvline(x=0.5, color='grey', linestyle='-', alpha=0.5)
ax.axvline(x=1.5, color='grey', linestyle='-', alpha=0.5)
ax.axvline(x=2.5, color='grey', linestyle='-', alpha=0.5)
# make x and y axis labels bigger
ax.xaxis.label.set_size(16)
ax.yaxis.label.set_size(16)

# put the legend on top of the plot
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.2), ncol=5, fancybox=True, shadow=True, fontsize=12)

plt.tight_layout()
plt.show()

# save figure
fig.savefig('abacus_energy_comparison_single_core.pdf', bbox_inches='tight')
# export data to csv
stats_no_baseline.to_csv('abacus_energy_comparison_single_core.csv', index=False)

# REGA at 1000 nRH normalized_energy for the geomean workload
print(stats_no_baseline[(stats_no_baseline['nrh'] == 1000) & (stats_no_baseline['config'] == 'REGA')]['normalized_energy'].mean()/stats_no_baseline[(stats_no_baseline['nrh'] == 1000) & (stats_no_baseline['config'] == 'ABACuS')]['normalized_energy'].mean())

#### Energy comparison (8-core)

In [ ]:
MY_MECHANISM_NAME = 'ABACuS'

### READ RESULTS INTO PANDAS DATAFRAME
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

resultsdir = "../results"
# list all directories in resultsdir
configs = [d for d in os.listdir(resultsdir) if os.path.isdir(os.path.join(resultsdir, d))]
configs = ['Baseline.yaml', 'MC-Baseline.yaml', 'Hydra-Baseline.yaml', 'REGA125.yaml', 'REGA250.yaml', 'REGA500.yaml', 'REGA1000.yaml', 'Graphene125.yaml', 'Graphene250.yaml', 'Graphene500.yaml', 'Graphene1000.yaml', 'Hydra125.yaml', 'Hydra250.yaml', 'Hydra500.yaml', 'Hydra1000.yaml', 'PARA125.yaml', 'PARA250.yaml', 'PARA500.yaml', 'PARA1000.yaml', 'ABACUS125.yaml', 'ABACUS250.yaml', 'ABACUS500.yaml', 'ABACUS1000.yaml']

# print found configs
print('Found configs: {}'.format(configs))
# list all directories under all configs
workloads = []
for c in configs:
    workloads.append([d for d in os.listdir(os.path.join(resultsdir, c)) if os.path.isdir(os.path.join(resultsdir, c, d))])
# find only the intersection of all workloads
# workloads = list(set.intersection(*map(set, workloads)))
workloads = list(set([item for sublist in workloads for item in sublist]))
# print found workloads
print('Found workloads: {}'.format(workloads))

stats_per_config_workload = []

# remove workloads with fewer than 7 dashes
workloads = [w for w in workloads if w.count('-') >= 7]

# for every config + workload directory
for c in configs:
    for w in workloads:
        # find all files in the directory
        if os.path.isdir(os.path.join(resultsdir, c, w)):
            files = [f for f in os.listdir(os.path.join(resultsdir, c, w)) if os.path.isfile(os.path.join(resultsdir, c, w, f))]
            # find the stats file
            stat_files = [f for f in files if f.endswith('.stats')]
            # if there is a stats file
            if stat_files:
                for stat_file in stat_files:
                    # if the stats_file has less than three lines skip it
                    if len(open(os.path.join(resultsdir, c, w, stat_file)).readlines()) < 3:
                        continue
                    
                    # print the name of the stats_file
                    print('Found stats file: {}'.format(os.path.join(os.path.join(resultsdir, c, w, stat_file))))

                    extension = ''
                    # if stats_file file name itself does not start with DDR4, parse it a bit
                    if not stat_file.startswith('DDR4'):
                        # get the config name from the stats_file name
                        extension = '_'.join(stat_file.split('_')[:-1])
                        # prepend underscore to extension
                        extension = '_' + extension

                    # read the stats file, name columns: 'name', 'value', 'description'
                    df = pd.read_csv(os.path.join(resultsdir, c, w, stat_file), header=None).T
                    df.columns = df.iloc[0]

                    # if df has this column total_dram_energy0_channel
                    if 'ramulator.total_dram_energy0_channel' in df.columns:
                        total_energy = df['ramulator.total_dram_energy0_channel'].values[1]
                    else:
                        total_energy = 0

                    # create a df with the config, workload and total_energy
                    df = pd.DataFrame({'config': [c], 'workload': [w], 'total_energy': [total_energy]})
                    df.reset_index(inplace=True, drop=True)
                    stats_per_config_workload.append(df)
            else:
                print('Config: {}, Workload: {}, Stats: No stats file found'.format(c, w))


# concatenate all stats into one dataframe
stats = pd.concat(stats_per_config_workload)


# remove workloads that have "gups" in them
stats = stats[~stats['workload'].str.contains('gups')]

# grep_map0 produced 0 cycles for any core except 7, weird bug, ignore for now
# stats = stats[~stats['workload'].str.contains('grep_map0')]

# also remove them from the list
workloads = [w for w in workloads if 'gups' not in w]

# grep_map0 produced 0 cycles for any core except 7, weird bug, ignore for now
# workloads = [w for w in workloads if 'grep_map0' not in w]


# replace all instances of "random_10.trace" in workload names with "gups"
stats['workload'] = stats['workload'].str.replace('random_10.trace', 'gups')
HIGH_RBMPKI = ['h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver', 'ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'stream_10.trace', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp']
stats = stats[stats['workload'].str.contains('|'.join(HIGH_RBMPKI))]
# remove h264_decode fro workloads
# stats = stats[~stats['workload'].str.contains('h264_decode')]
# remove h264_decode from HIGH_RBMPKI
# HIGH_RBMPKI.remove('h264_decode')

# remove from mc_only_stats the workloads that contain less than 7 dashes
stats = stats[stats['workload'].str.count('-') >= 7]

# print number of unique workloads
print('Unique workloads: {}'.format(stats['workload'].nunique()))

# keep mc_only_stats only for workloads that contain the strings in HIGH_RBMPKI

# remove "-16DR" from config names
stats['config'] = stats['config'].str.replace('-16DR', '')

# replace 1K with 1000 in config names
stats['config'] = stats['config'].str.replace('1K', '1000')

# replace 'Baseline' with 'Baseline0'
stats['config'] = stats['config'].str.replace('Baseline', 'Baseline0')

# add a new column that stores in integer the number in the config name
stats['nrh'] = stats['config'].str.extract('(\d+)').astype(int)

# remove numbers from config names
stats['config'] = stats['config'].str.replace('\d+', '', regex=True)

# remove yaml from config names
stats['config'] = stats['config'].str.replace('.yaml', '')

# replace SPR with APAR
stats['config'] = stats['config'].str.replace('ABACUS', MY_MECHANISM_NAME)

stats_copy = stats.copy()


# use seaborn-deep style
sns.set(font_scale=1.0)
sns.set_style("whitegrid")
sns.set_palette("pastel", n_colors=5)

stats = stats_copy.copy()

# copy the IPC of the baseline config as to all configs
baseline = stats[stats['config'] == 'Baseline']
baseline = baseline[['workload', 'total_energy']]
# baseline
baseline.columns = ['workload', 'baseline_energy']
stats = pd.merge(stats, baseline, on='workload')

#hydra baseline
hydra_baseline = stats[stats['config'] == 'Hydra-Baseline']
hydra_baseline = hydra_baseline[['workload', 'total_energy']]
# hydra_baseline
hydra_baseline.columns = ['workload', 'ramulator.hydra_baseline_energy']
stats = pd.merge(stats, hydra_baseline, on='workload')

stats['normalized_energy'] = stats['total_energy'] / stats['baseline_energy']

# remove normalized_energy below 1
stats = stats[stats['normalized_energy'] >= 0.9]
# remove 429.mcf
# stats = stats[~stats['workload'].str.contains('bfs_ny')]
# stats = stats[~stats['workload'].str.contains('bfs_cm2003')]

# normalized ipc for hydra is not correct, so we overwrite it with the correct value
stats.loc[stats['config'].str.contains('Hydra'), 'normalized_energy'] = stats['total_energy'] / stats['ramulator.hydra_baseline_energy']

# new dataframe that does not have the baseline configs
stats_no_baseline = stats[~stats['config'].str.contains('Baseline')]

# new dataframe that does not have the baseline configs
print(stats_no_baseline['config'].unique())

# order nRH from high to low
stats_no_baseline['nrh'] = pd.Categorical(stats_no_baseline['nrh'], categories=[1000, 500, 250, 125], ordered=True)

# order config in this order: abacus, Graphene, Hydra, REGA, PARA
stats_no_baseline['config'] = pd.Categorical(stats_no_baseline['config'], categories=['ABACuS', 'Graphene', 'Hydra', 'REGA', 'PARA'], ordered=True)

#boxplot of normalized IPC
fig, ax = plt.subplots(figsize=(10, 4))
# show mean values as well
ax = sns.boxplot(x="nrh", y="normalized_energy", hue="config", data=stats_no_baseline, showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black"})#, showfliers = False)
ax.set_xlabel('RowHammer Threshold ($N_{RH}$)')
ax.set_ylabel('Normalized Energy Distribution')
# draw a red line at y = 1.0, label it as baseline IPC
ax.axhline(y=1.0, color='r', linestyle='--')
# write above the red line 'baseline IPC'
ax.text(0.02, 0.02, 'Baseline DRAM energy', color='#e74c3c', transform=ax.transAxes, fontsize=15)
# extend the y axis to 1.2
ax.set_ylim(0.6, 4.0)
# color the 5th y tick red
ax.get_yticklabels()[1].set_color('#e74c3c')
# make axis tick font bigger
ax.tick_params(axis='both', which='major', labelsize=14)
# draw vertical lines to separate the rowhammer threshold values
ax.axvline(x=0.5, color='grey', linestyle='-', alpha=0.5)
ax.axvline(x=1.5, color='grey', linestyle='-', alpha=0.5)
ax.axvline(x=2.5, color='grey', linestyle='-', alpha=0.5)
# make x and y axis labels bigger
ax.xaxis.label.set_size(16)
ax.yaxis.label.set_size(16)

# put the legend on top of the plot
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.2), ncol=5, fancybox=True, shadow=True, fontsize=12)

plt.tight_layout()
plt.show()

# save figure
fig.savefig('abacus_energy_comparison_multi_core.pdf', bbox_inches='tight')
# export data to csv
stats_no_baseline.to_csv('abacus_energy_comparison_multi_core.csv', index=False)

# print mean DRAM energy for ABAcus at each nRH
print(stats_no_baseline[stats_no_baseline['config'] == 'ABACuS'].groupby('nrh')['normalized_energy'].mean())
print(stats_no_baseline[stats_no_baseline['config'] == 'ABACuS'].groupby('nrh')['normalized_energy'].max())

print(1-stats_no_baseline[(stats_no_baseline['config'] == 'ABACuS') & (stats_no_baseline['nrh'] == 125)]['normalized_energy'].mean()/stats_no_baseline[(stats_no_baseline['config'] == 'Hydra') & (stats_no_baseline['nrh'] == 125)]['normalized_energy'].mean())
print(1-stats_no_baseline[(stats_no_baseline['config'] == 'ABACuS') & (stats_no_baseline['nrh'] == 125)]['normalized_energy'].mean()/stats_no_baseline[(stats_no_baseline['config'] == 'REGA') & (stats_no_baseline['nrh'] == 125)]['normalized_energy'].mean())
print(1-stats_no_baseline[(stats_no_baseline['config'] == 'ABACuS') & (stats_no_baseline['nrh'] == 125)]['normalized_energy'].mean()/stats_no_baseline[(stats_no_baseline['config'] == 'PARA') & (stats_no_baseline['nrh'] == 125)]['normalized_energy'].mean())

print(1-stats_no_baseline[(stats_no_baseline['config'] == 'Graphene') & (stats_no_baseline['nrh'] == 125)]['normalized_energy'].mean()/stats_no_baseline[(stats_no_baseline['config'] == 'ABACuS') & (stats_no_baseline['nrh'] == 125)]['normalized_energy'].mean())


# print the number of unique workloads
print('Unique workloads: {}'.format(stats_no_baseline['workload'].nunique()))

### Adversarial workloads

In [ ]:
MY_MECHANISM_NAME = 'ABACuS'

### READ RESULTS INTO PANDAS DATAFRAME
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

resultsdir = "../results"
# list all directories in resultsdir
configs = [d for d in os.listdir(resultsdir) if os.path.isdir(os.path.join(resultsdir, d))]
configs = ['Baseline-AH.yaml', 'Baseline-AAH.yaml', 'Baseline-RH32.yaml', 'ABACUS500-RH32.yaml', 'ABACUS500-AH.yaml', 'ABACUS500-AAH.yaml', 'Graphene500-AH.yaml', 'Graphene500-AAH.yaml','Graphene500-RH32.yaml', 'REGA500-RH32.yaml', 'REGA500-AH.yaml', 'REGA500-AAH.yaml','PARA500-AH.yaml', 'PARA500-AAH.yaml', 'PARA500-RH32.yaml', 'Hydra500-AH.yaml', 'Hydra500-AAH.yaml','Hydra500-RH32.yaml', 'ABACUS500-RH32-Big.yaml', 'ABACUS500-AH-Big.yaml', 'ABACUS500-AAH-Big.yaml']
# print found configs
print('Found configs: {}'.format(configs))
# list all directories under all configs
workloads = []
for c in configs:
    workloads.append([d for d in os.listdir(os.path.join(resultsdir, c)) if os.path.isdir(os.path.join(resultsdir, c, d))])
# find only the intersection of all workloads
workloads = list(set.intersection(*map(set, workloads)))
# print found workloads
print('Found workloads: {}'.format(workloads))

stats_per_config_workload = []

# for every config + workload directory
for c in configs:
    for w in workloads:
        # find all files in the directory
        files = [f for f in os.listdir(os.path.join(resultsdir, c, w)) if os.path.isfile(os.path.join(resultsdir, c, w, f))]
        # find the stats file
        stat_files = [f for f in files if f.endswith('.stats')]
        # if there is a stats file
        if stat_files:
            for stat_file in stat_files:
                # if the stats_file has less than three lines skip it
                if len(open(os.path.join(resultsdir, c, w, stat_file)).readlines()) < 3:
                    continue
                
                # print the name of the stats_file
                print('Found stats file: {}'.format(os.path.join(os.path.join(resultsdir, c, w, stat_file))))

                extension = ''
                # if stats_file file name itself does not start with DDR4, parse it a bit
                if not stat_file.startswith('DDR4'):
                    # get the config name from the stats_file name
                    extension = '_'.join(stat_file.split('_')[:-1])
                    # prepend underscore to extension
                    extension = '_' + extension

                # read the stats file, name columns: 'name', 'value', 'description'
                df = pd.read_csv(os.path.join(resultsdir, c, w, stat_file), header=None).T
                df.columns = df.iloc[0]
                df.drop(0,inplace=True)
                # add a new column called 'config' with the config name
                df['config'] = c + extension
                # add a new column called 'workload' with the workload name
                df['workload'] = w
                # print the stats file
                # print('Config: {}, Workload: {}, Stats: {}'.format(c, w, df))
                # append the stats to the list
                df.reset_index(inplace=True, drop=True)
                stats_per_config_workload.append(df)
        else:
            print('Config: {}, Workload: {}, Stats: No stats file found'.format(c, w))

# concatenate all stats into one dataframe
stats = pd.concat(stats_per_config_workload)

# find elements where workload does not contain '-'
# these are multi core workloads
stats = stats[~stats['workload'].str.contains('-')]

# remove these two workloads: stream_10.trace and random_10.trace
stats = stats[~stats['workload'].isin(['gups'])]
# also from workloads
workloads = [w for w in workloads if not w in ['gups']]

# remove "-16DR" from config names
stats['config'] = stats['config'].str.replace('-16DR', '')

# replace 1K with 1000 in config names
stats['config'] = stats['config'].str.replace('1K', '1000')

# replace 'Baseline' with 'Baseline0'
stats['config'] = stats['config'].str.replace('Baseline', 'Baseline0')

# add a new column that stores in integer the number in the config name
stats['nrh'] = stats['config'].str.extract('(\d+)').astype(int)

# remove numbers from config names
stats['config'] = stats['config'].str.replace('\d+', '', regex=True)

# remove yaml from config names
stats['config'] = stats['config'].str.replace('.yaml', '')

# replace SPR with APAR
stats['config'] = stats['config'].str.replace('ABACUS', MY_MECHANISM_NAME)

stats.loc[stats['workload'] == 'random_10.trace', 'workload'] = 'gups'

# increasing order of rbmpki
# order = ['511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', '500.perlbench', '523.xalancbmk', '510.parest', '557.xz', '482.sphinx3', '505.mcf', '436.cactusADM', '471.omnetpp', '473.astar', '483.xalancbmk', '462.libquantum', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf']
order = ['h264_encode', '511.povray', '481.wrf', '541.leela', '538.imagick', '444.namd', '447.dealII', '464.h264ref', '456.hmmer', '403.gcc', '526.blender', '544.nab', '525.x264', '508.namd', 'grep_map0', '531.deepsjeng', '458.sjeng', '435.gromacs', '445.gobmk', '401.bzip2', '507.cactuBSSN', '502.gcc', 'ycsb_abgsave', 'tpch6', '500.perlbench', '523.xalancbmk', 'ycsb_dserver', 'ycsb_cserver', '510.parest', 'ycsb_bserver', 'ycsb_eserver', 'stream_10.trace', 'tpcc64', 'ycsb_aserver', '557.xz', '482.sphinx3', 'jp2_decode', '505.mcf', 'wc_8443', 'wc_map0', '436.cactusADM', '471.omnetpp', '473.astar', 'jp2_encode', 'tpch17', '483.xalancbmk', '462.libquantum', 'tpch2', '433.milc', '520.omnetpp', '437.leslie3d', '450.soplex', '459.GemsFDTD', '549.fotonik3d', '434.zeusmp', '519.lbm', '470.lbm', '429.mcf', 'gups', 'h264_decode', 'bfs_ny', 'bfs_cm2003', 'bfs_dblp']

# remove all workloads not in order
stats = stats[stats['workload'].isin(order)]
# also from the workload list
workloads = [w for w in workloads if w in order]

# order workloads according to the order
stats['workload'] = pd.Categorical(stats['workload'], categories=order, ordered=True)

stats_copy = stats.copy()

# use seaborn-deep style
sns.set(font_scale=1.0)
sns.set_style("whitegrid")
sns.set_palette("pastel", n_colors=4)

stats = stats_copy.copy()

# instructions per cycle (IPC) is record_cycles_insts_0 / record_cycs_core_0
stats['ramulator.ipc'] = stats['ramulator.record_insts_core_0'] / stats['ramulator.record_cycs_core_0']


stats['ramulator.rbmpki'] = (stats['ramulator.row_conflicts_channel_0_core'] + stats['ramulator.row_misses_channel_0_core']) /\
                            stats['ramulator.record_insts_core_0'] * 1000

# if ramulator.row_count_cache_evictions_channel_0 is zero, then make it 1 billion
stats.loc[stats['ramulator.row_count_cache_evictions_channel_0'] == 0, 'ramulator.row_count_cache_evictions_channel_0'] = 1000000000

stats['ramulator.rccepllcm'] = stats['ramulator.L3_cache_total_miss'] / stats['ramulator.row_count_cache_evictions_channel_0']


# copy the IPC of the baseline config as to all configs
baseline_ah = stats[stats['config'] == 'Baseline-AH']
baseline_ah = baseline_ah[['workload', 'ramulator.ipc']]
# baseline
baseline_ah.columns = ['workload', 'ramulator.baseline_ipc_ah']

baseline_rh = stats[stats['config'] == 'Baseline-RH']
baseline_rh = baseline_rh[['workload', 'ramulator.ipc']]
# baseline
baseline_rh.columns = ['workload', 'ramulator.baseline_ipc_rh']

baseline_aah = stats[stats['config'] == 'Baseline-AAH']
baseline_aah = baseline_aah[['workload', 'ramulator.ipc']]
# baseline
baseline_aah.columns = ['workload', 'ramulator.baseline_ipc_aah']

stats = pd.merge(stats, baseline_ah, on='workload')
stats = pd.merge(stats, baseline_rh, on='workload')
stats = pd.merge(stats, baseline_aah, on='workload')

stats['ramulator.normalized_ipc'] = stats['ramulator.ipc'] / stats['ramulator.baseline_ipc_rh']

stats.loc[stats['config'].str.contains('-AH'), 'ramulator.normalized_ipc'] = stats['ramulator.ipc'] / stats['ramulator.baseline_ipc_ah']
stats.loc[stats['config'].str.contains('-AAH'), 'ramulator.normalized_ipc'] = stats['ramulator.ipc'] / stats['ramulator.baseline_ipc_aah']



# add a new column called attack workload 
# if config contains '-RH' then the attack workload is RH
# if config contains '-AH' then the attack workload is AP
# if config contains '-AAH' then the attack workload is HAP

stats['attack_workload'] = 'RowHammer Attack'
stats.loc[stats['config'].str.contains('-AH'), 'attack_workload'] = 'Hydra-Adversarial'
stats.loc[stats['config'].str.contains('-AAH'), 'attack_workload'] = 'ABACuS-Adversarial'

# order attack_workload
stats['attack_workload'] = pd.Categorical(stats['attack_workload'], categories=['RowHammer Attack', 'Hydra-Adversarial', 'ABACuS-Adversarial'], ordered=True)

# remove -RH -AH -AAH from config
stats['config'] = stats['config'].str.replace('-RH', '')
stats['config'] = stats['config'].str.replace('-AH', '')
stats['config'] = stats['config'].str.replace('-AAH', '')


# remove baseline
stats = stats[~stats['config'].str.contains('Baseline')]

# order configs
stats['config'] = pd.Categorical(stats['config'], categories=['ABACuS', 'Graphene', 'Hydra', 'REGA', 'PARA'], ordered=True)

# if config is Hydra and attack_workload is Hydra Adversarial Pattern AP then

#boxplot of normalized IPC
fig, ax = plt.subplots(figsize=(10, 4))
# show mean values as well
ax = sns.boxplot(x="attack_workload", y="ramulator.normalized_ipc", hue="config", data=stats, showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black"})
ax.set_xlabel('')
ax.set_ylabel('Normalized IPC Distribution')
# draw a red line at y = 1.0, label it as baseline IPC
ax.axhline(y=1.0, color='r', linestyle='--')
# write above the red line 'baseline IPC'
ax.text(0.02, 0.93, 'Baseline IPC', color='#e74c3c', transform=ax.transAxes, fontsize=15)
# extend the y axis to 1.2
ax.set_ylim(0, 1.1)
# color the 5th y tick red
ax.get_yticklabels()[5].set_color('#e74c3c')
# make axis tick font bigger
ax.tick_params(axis='both', which='major', labelsize=14)
# draw vertical lines to separate the rowhammer threshold values
ax.axvline(x=0.5, color='grey', linestyle='-', alpha=0.5)
ax.axvline(x=1.5, color='grey', linestyle='-', alpha=0.5)
ax.axvline(x=2.5, color='grey', linestyle='-', alpha=0.5)
# make x and y axis labels bigger
ax.xaxis.label.set_size(16)
ax.yaxis.label.set_size(16)

# put the legend on top of the plot
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=20, fancybox=True, shadow=True, fontsize=10)

plt.tight_layout()
plt.show()

# save figure
fig.savefig('abacus_adversarial.pdf', bbox_inches='tight')

# use seaborn-deep style
sns.set(font_scale=1.0)
sns.set_style("whitegrid")
sns.set_palette("pastel", n_colors=4)

stats = stats_copy.copy()

# instructions per cycle (IPC) is record_cycles_insts_0 / record_cycs_core_0
stats['ramulator.ipc'] = stats['ramulator.record_insts_core_0'] / stats['ramulator.record_cycs_core_0']


stats['ramulator.rbmpki'] = (stats['ramulator.row_conflicts_channel_0_core'] + stats['ramulator.row_misses_channel_0_core']) /\
                            stats['ramulator.record_insts_core_0'] * 1000

# if ramulator.row_count_cache_evictions_channel_0 is zero, then make it 1 billion
stats.loc[stats['ramulator.row_count_cache_evictions_channel_0'] == 0, 'ramulator.row_count_cache_evictions_channel_0'] = 1000000000

stats['ramulator.rccepllcm'] = stats['ramulator.L3_cache_total_miss'] / stats['ramulator.row_count_cache_evictions_channel_0']


# copy the IPC of the baseline config as to all configs
baseline_ah = stats[stats['config'] == 'Baseline-AH']
baseline_ah = baseline_ah[['workload', 'ramulator.ipc']]
# baseline
baseline_ah.columns = ['workload', 'ramulator.baseline_ipc_ah']

baseline_rh = stats[stats['config'] == 'Baseline-RH']
baseline_rh = baseline_rh[['workload', 'ramulator.ipc']]
# baseline
baseline_rh.columns = ['workload', 'ramulator.baseline_ipc_rh']

baseline_aah = stats[stats['config'] == 'Baseline-AAH']
baseline_aah = baseline_aah[['workload', 'ramulator.ipc']]
# baseline
baseline_aah.columns = ['workload', 'ramulator.baseline_ipc_aah']

stats = pd.merge(stats, baseline_ah, on='workload')
stats = pd.merge(stats, baseline_rh, on='workload')
stats = pd.merge(stats, baseline_aah, on='workload')

stats['ramulator.normalized_ipc'] = stats['ramulator.ipc'] / stats['ramulator.baseline_ipc_rh']

stats.loc[stats['config'].str.contains('-AH'), 'ramulator.normalized_ipc'] = stats['ramulator.ipc'] / stats['ramulator.baseline_ipc_ah']
stats.loc[stats['config'].str.contains('-AAH'), 'ramulator.normalized_ipc'] = stats['ramulator.ipc'] / stats['ramulator.baseline_ipc_aah']



# add a new column called attack workload 
# if config contains '-RH' then the attack workload is RH
# if config contains '-AH' then the attack workload is AP
# if config contains '-AAH' then the attack workload is HAP

stats['attack_workload'] = 'RowHammer Attack'
stats.loc[stats['config'].str.contains('-AH'), 'attack_workload'] = 'Hydra-Adversarial'
stats.loc[stats['config'].str.contains('-AAH'), 'attack_workload'] = 'ABACuS-Adversarial'

# order attack_workload
stats['attack_workload'] = pd.Categorical(stats['attack_workload'], categories=['RowHammer Attack', 'Hydra-Adversarial', 'ABACuS-Adversarial'], ordered=True)

# remove -RH -AH -AAH from config
stats['config'] = stats['config'].str.replace('-RH', '')
stats['config'] = stats['config'].str.replace('-AH', '')
stats['config'] = stats['config'].str.replace('-AAH', '')


# remove baseline
stats = stats[~stats['config'].str.contains('Baseline')]

# order configs
stats['config'] = pd.Categorical(stats['config'], categories=['ABACuS', 'ABACuS-Big'], ordered=True)

# if config is Hydra and attack_workload is Hydra Adversarial Pattern AP then

#boxplot of normalized IPC
fig, ax = plt.subplots(figsize=(7, 2))
# show mean values as well
ax = sns.boxplot(x="attack_workload", y="ramulator.normalized_ipc", hue="config", data=stats, showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black"})
ax.set_xlabel('')
ax.set_ylabel('Normalized IPC\nDistribution')
# draw a red line at y = 1.0, label it as baseline IPC
ax.axhline(y=1.0, color='r', linestyle='--')
# write above the red line 'baseline IPC'
ax.text(0.02, 0.82, 'Baseline IPC', color='#e74c3c', transform=ax.transAxes, fontsize=15)
# extend the y axis to 1.2
ax.set_ylim(0.7, 1.1)
# color the 5th y tick red
ax.get_yticklabels()[3].set_color('#e74c3c')
# make axis tick font bigger
ax.tick_params(axis='both', which='major', labelsize=14)
# draw vertical lines to separate the rowhammer threshold values
ax.axvline(x=0.5, color='grey', linestyle='-', alpha=0.5)
ax.axvline(x=1.5, color='grey', linestyle='-', alpha=0.5)
ax.axvline(x=2.5, color='grey', linestyle='-', alpha=0.5)
# make x and y axis labels bigger
ax.xaxis.label.set_size(12)
ax.yaxis.label.set_size(16)

# maxe x axis labels smaller
ax.xaxis.set_tick_params(labelsize=12)

# put the legend on top of the plot
ax.legend(loc='upper center', bbox_to_anchor=(0.65, 1.15), ncol=20, fancybox=True, shadow=True, fontsize=12)

plt.tight_layout()
plt.show()

# save figure
fig.savefig('abacus_adversarial_big.pdf', bbox_inches='tight')


# print mean normalized ipc of ABACUS in rowhammer attack
print('ABACuS mean normalized IPC in RowHammer attack: ', 1-(stats.loc[(stats['config'] == 'ABACuS') & (stats['attack_workload'] == 'RowHammer Attack'), 'ramulator.normalized_ipc'].mean()))
print('ABACuS-Big mean normalized IPC in RowHammer attack: ', 1-(stats.loc[(stats['config'] == 'ABACuS-Big') & (stats['attack_workload'] == 'RowHammer Attack'), 'ramulator.normalized_ipc'].mean()))


print('ABACuS mean normalized IPC in HydraADV: ', 1-(stats.loc[(stats['config'] == 'ABACuS') & (stats['attack_workload'] == 'Hydra-Adversarial'), 'ramulator.normalized_ipc'].mean()))
print('ABACuS-Big mean normalized IPC in HydraADV: ', 1-(stats.loc[(stats['config'] == 'ABACuS-Big') & (stats['attack_workload'] == 'Hydra-Adversarial'), 'ramulator.normalized_ipc'].mean()))


print('ABACuS mean normalized IPC in ABACuSADV: ', 1-(stats.loc[(stats['config'] == 'ABACuS') & (stats['attack_workload'] == 'ABACuS-Adversarial'), 'ramulator.normalized_ipc'].mean()))
print('ABACuS-Big mean normalized IPC in ABACuSADV: ', 1-(stats.loc[(stats['config'] == 'ABACuS-Big') & (stats['attack_workload'] == 'ABACuS-Adversarial'), 'ramulator.normalized_ipc'].mean()))

#### TRR Analysis Plot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

counters = [16, 32, 64, 96, 128, 164]
N_RH = [70070, 35490, 17290, 10920, 8190, 6370]

# whitegrid style
sns.set_style("whitegrid")

# colorpalette with 6 colors
# first color is blue
# other colors are red
colors = sns.color_palette(sns.color_palette('Blues', 1) + 5 * sns.color_palette('Greens', 1))

myblue = sns.color_palette('Blues', 1)[0]
mygreen = sns.color_palette('Greens', 1)[0]

plt.figure(figsize=(5, 3))

# plot n_rh by counters as barplot
sns.barplot(x=counters, y=N_RH, palette=colors)

# make bars thinner and add black border
for patch in plt.gca().patches:
    patch.set_linewidth(1.0)
    patch.set_edgecolor('black')

# draw a horizontal line at y = 4800
plt.axhline(y=4800, color='red', linestyle='--')

# add a y axis label at 4800 (modify axis labels) NOT BY ADDING TEXT TO THE PLOT
plt.yticks([4800] + list(range(10000, 80000, 10000)), ['4.8K'] + list(str(x) + 'K' for x in range(10, 80, 10)))

# y axis label is $N_{RH}$
plt.ylabel('RowHammer Threshold ($N_{RH}$)', fontsize=12)

# x axis label is Number of counters
plt.xlabel('Number of counters', fontsize=12)

# increase font isze of x and y axis labels
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)

# annotate the horizontal red line "Minimum observed RowHammer Threshold"
plt.annotate('', xy=(0.7, 4800), xytext=(0.7, 15000),
             arrowprops=dict(facecolor='red', shrink=0.05, width=1, headwidth=5,edgecolor='red'),
             horizontalalignment='center')

# add text inside a semi-transparent box
plt.text(0.21, 0.3, 'Minimum observed\nRowHammer Threshold', size=8, weight='bold', color='red', horizontalalignment='center', verticalalignment='center',
         transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.9, edgecolor='red'))

plt.text(0.211, 0.8, 'Reverse engineered (in [55]) TRR configuration', weight='bold', size=8, color=myblue, horizontalalignment='left', verticalalignment='center',
         transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.9, edgecolor=myblue))

plt.text(0.30, 0.66, 'Future configurations with more counters', weight='bold', size=8, color=mygreen, horizontalalignment='left', verticalalignment='center',
         transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.9, edgecolor=mygreen))

# save figure to pdf 'trr-thresholds.pdf'
plt.savefig('trr-thresholds.pdf', bbox_inches='tight')

# above annotation but text centered
# plt.annotate('Minimum observed\nRowHammer Threshold', xy=(0, 4800), xytext=(1, 45000),
#              arrowprops=dict(facecolor='red', shrink=0.05, width=1, headwidth=5,edgecolor='red'),
#              horizontalalignment='center')